<a href="https://colab.research.google.com/github/Renlim61/MVP_Product001_2025_Tier120pbc/blob/version-history/Phase1_RAG_MVP_v16_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================
# Colab setup
# ==============================================================
%pip install -q faiss-cpu openai gradio PyPDF2 python-docx


In [2]:
# -*- coding: utf-8 -*-
"""Phase1_RAG_MVP_v16_1.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1p5oRzSNrrsgLwxdyvPBohruPzRimlXYO
"""

# Commented out IPython magic to ensure Python compatibility.
# -*- coding: utf-8 -*-
"""RAG_MVP_v16_Prod_Ready.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1HeWDkcsep3wz8eF92iUeDD1P9ecBpDlw
"""

# -*- coding: utf-8 -*-
"""RAG_MVP_v16_login_logout_stable.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/13hSYOl6kd3G1xwcYnIGtXN0TMlS-jt0e
"""
#
# Commented out IPython magic to ensure Python compatibility.

# Commented out IPython magic to ensure Python compatibility.
# CELL 0 / STEP 0 – Install & Imports


'RAG_MVP_v16_login_logout_stable.ipynb\n\nAutomatically generated by Colab.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/13hSYOl6kd3G1xwcYnIGtXN0TMlS-jt0e\n'

In [3]:
# ============================================================

# Run this once at the top of the notebook (Colab style).
#=============================================================
# %pip install -q faiss-cpu openai gradio PyPDF2 python-docx

import os
import io
import sys
import time
import json
import shutil
import pickle
import sqlite3
from uuid import uuid4
from datetime import datetime, timedelta, timezone
from typing import List, Dict, Any, Tuple, Optional

import numpy as np
import faiss
import gradio as gr

from PyPDF2 import PdfReader
from docx import Document as DocxDocument

# Global datetime import for the whole notebook
from datetime import datetime, timezone, timedelta

# Colab detection
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive as colab_drive

# OpenAI client (v1 library)
from openai import OpenAI

# CELL 1 — Execution flags & persistence-safe paths (Colab/Drive)
#============================================================
# If True, we only run the DB pre-flight checks and stop BEFORE launching Gradio.
RUN_PREFLIGHT_ONLY = False

import os



In [4]:
# ============================================================

# PERSISTENCE CONFIG (Colab-safe)


In [5]:
# ============================================================

# Colab runtime storage under /content is ephemeral. Persist DB + indexes to Drive.
USE_DRIVE_PERSISTENCE = True          # Set False only if running locally
DRIVE_SUBFOLDER = "rag_mvp"           # Folder under MyDrive

# If IN_COLAB is not defined for any reason, define it safely.
try:
    IN_COLAB
except NameError:
    try:
        import google.colab  # type: ignore
        IN_COLAB = True
    except Exception:
        IN_COLAB = False

if IN_COLAB and USE_DRIVE_PERSISTENCE:
    try:
        from google.colab import drive as colab_drive  # type: ignore
        colab_drive.mount("/content/drive", force_remount=False)
    except Exception as e:
        print("⚠️ Drive mount issue (continuing):", e)

    BASE_DIR = f"/content/drive/MyDrive/{DRIVE_SUBFOLDER}"
else:
    BASE_DIR = os.path.join(os.getcwd(), DRIVE_SUBFOLDER)

os.makedirs(BASE_DIR, exist_ok=True)

# Persistent DB path
DB_PATH = os.path.join(BASE_DIR, "rag_documents.db")

# Persistent index/log directories
INDEX_DIR = os.path.join(BASE_DIR, "indexes")
os.makedirs(INDEX_DIR, exist_ok=True)

TRACE_LOG_PATH = os.path.join(BASE_DIR, "debug_trace_v16.log")

print("PERSISTENCE CONFIG")
print("IN_COLAB:", IN_COLAB)
print("BASE_DIR:", BASE_DIR)
print("DB_PATH :", DB_PATH)
print("INDEX_DIR:", INDEX_DIR)
print("TRACE_LOG_PATH:", TRACE_LOG_PATH)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PERSISTENCE CONFIG
IN_COLAB: True
BASE_DIR: /content/drive/MyDrive/rag_mvp
DB_PATH : /content/drive/MyDrive/rag_mvp/rag_documents.db
INDEX_DIR: /content/drive/MyDrive/rag_mvp/indexes
TRACE_LOG_PATH: /content/drive/MyDrive/rag_mvp/debug_trace_v16.log


In [6]:
# ============================================================


# CELL 1.75 — Global Configuration Constants


In [7]:
# ============================================================


# -------- Chunking --------
CHUNK_SIZE = 800          # tokens/characters per chunk (adjust as needed)
CHUNK_OVERLAP = 100       # overlap between chunks

# -------- Embeddings / Index --------
EMBEDDING_DIM = 1536      # OpenAI text-embedding-3-small
DEFAULT_EMBED_MODEL = "text-embedding-3-small"

# -------- Retrieval --------
TOP_K = 5

# -------- Limits --------
MAX_FILES_PER_UPLOAD = 20
MAX_FILE_SIZE_MB = 25

print("CONFIG CONSTANTS LOADED")
print("CHUNK_SIZE =", CHUNK_SIZE)
print("CHUNK_OVERLAP =", CHUNK_OVERLAP)
print("DEFAULT_EMBED_MODEL =", DEFAULT_EMBED_MODEL)



CONFIG CONSTANTS LOADED
CHUNK_SIZE = 800
CHUNK_OVERLAP = 100
DEFAULT_EMBED_MODEL = text-embedding-3-small


In [8]:
# ============================================================


# CELL 1.9 — OpenAI Client Factory


In [9]:
# ============================================================


import os

def build_openai_client(api_key: str):
    """
    Returns an OpenAI client using the new SDK interface.
    Centralized so ingestion and chat use the same construction.
    """
    if not api_key:
        raise ValueError("OpenAI API key is required.")

    try:
        from openai import OpenAI
    except ImportError as e:
        raise RuntimeError("openai package not installed.") from e

    return OpenAI(api_key=api_key)

print("✅ build_openai_client() is defined.")



✅ build_openai_client() is defined.


In [10]:
# ============================================================


# CELL 1.95 — Model Defaults + Resolver


In [11]:
# ============================================================


# Defaults (used across embed + chat if user does not override)
EMBED_MODEL_DEFAULT = "text-embedding-3-small"
CHAT_MODEL_DEFAULT  = "gpt-4.1-mini"

def resolve_models(chat_model: str | None, embed_model: str | None) -> tuple[str, str]:
    """
    Normalize selected models and apply defaults.
    Returns (resolved_chat_model, resolved_embed_model).
    """
    resolved_chat = (chat_model or "").strip() or CHAT_MODEL_DEFAULT
    resolved_embed = (embed_model or "").strip() or EMBED_MODEL_DEFAULT
    return resolved_chat, resolved_embed

print("✅ resolve_models() is defined.")
print("   CHAT_MODEL_DEFAULT =", CHAT_MODEL_DEFAULT)
print("   EMBED_MODEL_DEFAULT =", EMBED_MODEL_DEFAULT)

# CELL 1.5 / STEP 1.5 – Validate OpenAI Key & Models


✅ resolve_models() is defined.
   CHAT_MODEL_DEFAULT = gpt-4.1-mini
   EMBED_MODEL_DEFAULT = text-embedding-3-small


In [12]:
# ============================================================


def validate_openai_key_and_models(api_key: str, chat_model: str, embed_model: str) -> str:
    """
    Lightweight validation:
    - Instantiate client
    - Do a tiny chat completion
    - Do a small embedding call
    Returns a human-readable status string.
    """
    if not api_key:
        return "❌ Please provide an OpenAI API key."

    try:
        client = build_openai_client(api_key)
        resolved_chat, resolved_embed = resolve_models(chat_model, embed_model)

        # Tiny chat test
        _ = client.chat.completions.create(
            model=resolved_chat,
            messages=[
                {"role": "system", "content": "Model availability test."},
                {"role": "user", "content": "Respond with 'OK' only."},
            ],
            max_tokens=2,
            temperature=0.0,
        )

        # Tiny embedding test
        _ = client.embeddings.create(
            model=resolved_embed,
            input=["test"],
        )

        return f"✅ OpenAI key valid. Chat model: `{resolved_chat}`, Embed model: `{resolved_embed}`"

    except Exception as e:
        return f"❌ Error validating key/models: {e}"



In [13]:
# ============================================================


#============================================================

# CELL 1.25 — Soft DB sanity gate (bootstrap + report)
#============================================================
import sqlite3
import time

REQUIRED_TABLES = [
    "documents",
    "cohort_docs",
    "cohorts",
    "cohort_meta",
    "users",
    "chat_history",
    "audit_log",
    "model_registry",
]

def soft_db_sanity_gate(db_path: str, base_dir: str, required_tables: list[str]) -> None:
    print("="*80)
    print("DB SANITY GATE — PRE-FLIGHT CHECK")
    print("="*80)
    print("BASE_DIR:", base_dir)
    print("DB_PATH :", db_path)

    os.makedirs(os.path.dirname(db_path), exist_ok=True)

    # Create DB file if missing (do NOT fail on first run)
    first_create = not os.path.exists(db_path)
    if first_create:
        # Touch/create by connecting
        conn = sqlite3.connect(db_path)
        conn.commit()
        conn.close()

    # Bootstrap minimal schema so the app can start from a clean environment
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # Minimal table definitions (aligned with MVP expectations)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS documents (
            id              TEXT PRIMARY KEY,
            doc_name        TEXT NOT NULL,
            cohort_name     TEXT NOT NULL,
            index_id        TEXT NOT NULL,
            n_chunks        INTEGER NOT NULL,
            embed_model     TEXT NOT NULL,
            created_at      TEXT NOT NULL
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS cohort_docs (
            cohort_name TEXT NOT NULL,
            doc_name    TEXT NOT NULL
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS cohorts (
            id              INTEGER PRIMARY KEY AUTOINCREMENT,
            name            TEXT NOT NULL UNIQUE,
            description     TEXT,
            owner_user_id   TEXT,
            created_at      TEXT NOT NULL,
            updated_at      TEXT NOT NULL
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS cohort_meta (
            cohort_name    TEXT PRIMARY KEY,
            owner_user_id  TEXT,
            is_shared      INTEGER DEFAULT 0,
            allow_clone    INTEGER DEFAULT 0,
            created_ts     TEXT
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS users (
            user_id      TEXT PRIMARY KEY,
            display_name TEXT,
            role         TEXT,
            created_at   TEXT NOT NULL
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS chat_history (
            id             INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id        TEXT,
            role           TEXT,
            cohort_name    TEXT,
            original_query TEXT,
            improved_query TEXT,
            which_prompt   TEXT,
            answer         TEXT,
            chat_model     TEXT,
            created_at     TEXT NOT NULL
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS audit_log (
            id        INTEGER PRIMARY KEY AUTOINCREMENT,
            ts        TEXT NOT NULL,
            username  TEXT,
            role      TEXT,
            action    TEXT NOT NULL,
            details   TEXT
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS model_registry (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            provider TEXT,
            model_id TEXT,
            display_name TEXT,
            model_type TEXT,
            enabled INTEGER DEFAULT 1,
            is_default INTEGER DEFAULT 0,
            cost_score INTEGER DEFAULT 2,
            latency_score INTEGER DEFAULT 2,
            max_context_tokens INTEGER,
            api_base TEXT,
            notes TEXT
        )
    """)

    conn.commit()

    # Inspect tables
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
    tables = [r[0] for r in cur.fetchall()]
    missing = [t for t in required_tables if t not in set(tables)]

    # File stats
    exists = os.path.exists(db_path)
    size = os.path.getsize(db_path) if exists else None
    mtime = time.ctime(os.path.getmtime(db_path)) if exists else None

    print("Exists  :", exists)
    print("Size    :", size)
    print("Modified:", mtime)
    print("Tables  :", tables)

    if missing:
        print("❌ DB SANITY GATE FAILED — still missing required tables:", missing)
        raise RuntimeError(f"DB schema incomplete after bootstrap. Missing tables: {missing}")

    print("✅ DB SANITY GATE PASSED")
    print("="*80)

    conn.close()

# Run the gate
soft_db_sanity_gate(DB_PATH, BASE_DIR, REQUIRED_TABLES)

# Optional stop (without raising a traceback)
STOP_AFTER_PREFLIGHT = bool(RUN_PREFLIGHT_ONLY)
if STOP_AFTER_PREFLIGHT:
    print("Stopping after pre-flight by configuration (RUN_PREFLIGHT_ONLY=True).")

# CELL 2 / STEP 2 – Document Loading & Chunking


DB SANITY GATE — PRE-FLIGHT CHECK
BASE_DIR: /content/drive/MyDrive/rag_mvp
DB_PATH : /content/drive/MyDrive/rag_mvp/rag_documents.db
Exists  : True
Size    : 94208
Modified: Thu Dec 18 19:27:47 2025
Tables  : ['audit_log', 'chat_history', 'cohort_docs', 'cohort_meta', 'cohorts', 'demo_prompts', 'documents', 'model_registry', 'sqlite_sequence', 'users']
✅ DB SANITY GATE PASSED


In [14]:
# ============================================================


def load_pdf(file_bytes: bytes) -> str:
    reader = PdfReader(io.BytesIO(file_bytes))
    texts = []
    for page in reader.pages:
        try:
            txt = page.extract_text() or ""
        except Exception:
            txt = ""
        texts.append(txt)
    return "\n".join(texts)

def load_docx(file_bytes: bytes) -> str:
    f = io.BytesIO(file_bytes)
    doc = DocxDocument(f)
    return "\n".join(p.text for p in doc.paragraphs)

def load_txt(file_bytes: bytes, encoding: str = "utf-8") -> str:
    return file_bytes.decode(encoding, errors="ignore")

def load_file_to_text(file_obj) -> Tuple[str, str]:
    """
    Accepts either:
    - a string filepath (when gr.File(type="filepath") is used), or
    - a file-like object with a .name attribute (older behavior).

    Returns:
      (text_content, original_filename)
    """
    # Case 1: gr.File(type="filepath") -> we get a string path
    if isinstance(file_obj, str):
        path = file_obj
        name = os.path.basename(path)
    else:
        # Case 2: some object with a .name attribute
        path = getattr(file_obj, "name", None)
        if path is None:
            raise ValueError("Unsupported file object from uploader.")
        name = os.path.basename(path)

    with open(path, "rb") as f:
        data = f.read()

    lower = name.lower()
    if lower.endswith(".pdf"):
        text = load_pdf(data)
    elif lower.endswith(".docx"):
        text = load_docx(data)
    elif lower.endswith(".txt"):
        text = load_txt(data)
    else:
        raise ValueError(f"Unsupported file type for {name}")

    return text, name


def chunk_text(text: str, chunk_size: int = CHUNK_SIZE, overlap: int = CHUNK_OVERLAP) -> List[str]:
    """
    Simple sliding-window chunking.
    """
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    tokens = text.split()
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk_tokens = tokens[start:end]
        chunk = " ".join(chunk_tokens)
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks



In [15]:
# ============================================================


# CELL 3 / STEP 3 – Embedding & FAISS Index Helpers


In [16]:
# ============================================================


def embed_texts(
    api_key: str,
    embed_model: str,
    texts: List[str],
    batch_size: int = 32,
) -> np.ndarray:
    """
    Embed a list of texts using OpenAI embeddings.
    Returns an ndarray of shape (N, D).
    """
    client = build_openai_client(api_key)
    resolved_chat, resolved_embed = resolve_models("", embed_model)

    vectors: List[List[float]] = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        resp = client.embeddings.create(model=resolved_embed, input=batch)
        for d in resp.data:
            vectors.append(d.embedding)

    arr = np.array(vectors, dtype="float32")
    return arr

def build_faiss_index(vectors: np.ndarray) -> faiss.IndexFlatIP:
    """
    Build a simple inner-product FAISS index from vectors.
    """
    norm = np.linalg.norm(vectors, axis=1, keepdims=True) + 1e-10
    normed = vectors / norm
    dim = normed.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(normed)
    return index

def save_index(index: faiss.IndexFlatIP, index_id: str):
    path = os.path.join(INDEX_DIR, f"{index_id}.faiss")
    faiss.write_index(index, path)

def load_index(index_id: str) -> faiss.IndexFlatIP:
    path = os.path.join(INDEX_DIR, f"{index_id}.faiss")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Index file not found: {path}")
    return faiss.read_index(path)

def save_metadata(index_id: str, meta: Dict[str, Any]):
    path = os.path.join(INDEX_DIR, f"{index_id}.pkl")
    with open(path, "wb") as f:
        pickle.dump(meta, f)

def load_metadata(index_id: str) -> Dict[str, Any]:
    path = os.path.join(INDEX_DIR, f"{index_id}.pkl")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Metadata file not found: {path}")
    with open(path, "rb") as f:
        return pickle.load(f)



In [17]:
# ============================================================


# CELL 4 / STEP 4 – SQLite Persistence for Documents & Cohorts


In [18]:
# ============================================================

import os
import sqlite3
from uuid import uuid4

assert "BASE_DIR" in globals(), "BASE_DIR is not defined. Run CELL 1 first."
assert "DB_PATH"  in globals(), "DB_PATH is not defined. Run CELL 1 first."
print("STEP 4 starting with DB_PATH:", DB_PATH)

def get_db_conn():
    # Ensure BASE_DIR exists (defined in CELL 1)
    os.makedirs(BASE_DIR, exist_ok=True)
    # Ensure DB folder exists
    os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)
    return sqlite3.connect(DB_PATH)



def ensure_docs_table():
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS documents (
            id              TEXT PRIMARY KEY,
            doc_name        TEXT NOT NULL,
            cohort_name     TEXT NOT NULL,
            index_id        TEXT NOT NULL,
            n_chunks        INTEGER NOT NULL,
            embed_model     TEXT NOT NULL,
            created_at      TEXT NOT NULL
        )
        """
    )
    conn.commit()
    conn.close()


def ensure_cohort_table():
    """
    Ensure both:
      - cohorts: cohort metadata (required by v16 tests)
      - cohort_docs: mapping of cohort_name -> doc_name (used by the app)
    """
    conn = get_db_conn()
    cur = conn.cursor()

    # ---- New table required by tests ----
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS cohorts (
            id              INTEGER PRIMARY KEY AUTOINCREMENT,
            name            TEXT NOT NULL UNIQUE,
            description     TEXT,
            owner_user_id   TEXT,
            created_at      TEXT NOT NULL,
            updated_at      TEXT NOT NULL
        )
        """
    )

    # ---- Existing mapping table (unchanged behavior) ----
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS cohort_docs (
            cohort_name TEXT NOT NULL,
            doc_name    TEXT NOT NULL
        )
        """
    )

    conn.commit()
    conn.close()


def list_cohorts() -> List[str]:
    # We keep existing behavior: list distinct names from cohort_docs
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute("SELECT DISTINCT cohort_name FROM cohort_docs ORDER BY cohort_name ASC")
    rows = cur.fetchall()
    conn.close()
    return [r[0] for r in rows]


def list_docs_in_cohort(cohort_name: str) -> List[str]:
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        "SELECT doc_name FROM cohort_docs WHERE cohort_name = ? ORDER BY doc_name ASC",
        (cohort_name,),
    )
    rows = cur.fetchall()
    conn.close()
    return [r[0] for r in rows]


def add_docs_to_cohort(cohort_name: str, doc_names: List[str]):
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()
    for dn in doc_names:
        cur.execute(
            "INSERT INTO cohort_docs (cohort_name, doc_name) VALUES (?, ?)",
            (cohort_name, dn),
        )
    conn.commit()
    conn.close()


def rename_cohort(old_name: str, new_name: str):
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        "UPDATE cohort_docs SET cohort_name = ? WHERE cohort_name = ?",
        (new_name, old_name),
    )
    cur.execute(
        "UPDATE documents SET cohort_name = ? WHERE cohort_name = ?",
        (new_name, old_name),
    )
    conn.commit()
    conn.close()


def delete_cohort(cohort_name: str, reassign_to: Optional[str] = None) -> str:
    """
    Delete a cohort. If reassign_to is provided, documents move there.
    Otherwise, documents are deleted (and their indexes removed).
    NOTE: This is intentionally explicit to avoid orphaned docs.
    """
    ensure_docs_table()
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()

    cur.execute(
        "SELECT id, index_id FROM documents WHERE cohort_name = ?",
        (cohort_name,),
    )
    docs = cur.fetchall()

    if reassign_to:
        # Just move docs
        cur.execute(
            "UPDATE documents SET cohort_name = ? WHERE cohort_name = ?",
            (reassign_to, cohort_name),
        )
        cur.execute(
            "UPDATE cohort_docs SET cohort_name = ? WHERE cohort_name = ?",
            (reassign_to, cohort_name),
        )
        msg = f"✅ Cohort '{cohort_name}' renamed/reassigned to '{reassign_to}'. No indexes deleted."
    else:
        # Delete docs and indexes
        for doc_id, index_id in docs:
            # Remove index & metadata
            faiss_path = os.path.join(INDEX_DIR, f"{index_id}.faiss")
            pkl_path = os.path.join(INDEX_DIR, f"{index_id}.pkl")
            for p in [faiss_path, pkl_path]:
                if os.path.exists(p):
                    os.remove(p)
            cur.execute("DELETE FROM documents WHERE id = ?", (doc_id,))

        cur.execute("DELETE FROM cohort_docs WHERE cohort_name = ?", (cohort_name,))
        msg = f"✅ Cohort '{cohort_name}' and its documents/indexes were deleted."

    conn.commit()
    conn.close()
    return msg


def register_document(
    doc_name: str,
    cohort_name: str,
    index_id: str,
    n_chunks: int,
    embed_model: str,
):
    ensure_docs_table()
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()
    doc_id = str(uuid4())

    # ✅ Use dt.datetime (module alias) so the bottom `import datetime`
    #    in the self-test cell cannot break this.
    created_at = dt.datetime.now(dt.timezone.utc).isoformat()

    cur.execute(
        """
        INSERT INTO documents (id, doc_name, cohort_name, index_id, n_chunks,
                               embed_model, created_at)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """,
        (doc_id, doc_name, cohort_name, index_id, n_chunks, embed_model, created_at),
    )
    cur.execute(
        "INSERT INTO cohort_docs (cohort_name, doc_name) VALUES (?, ?)",
        (cohort_name, doc_name),
    )
    conn.commit()
    conn.close()
    return doc_id


def get_doc_index_id(doc_name: str, cohort_name: str) -> Optional[str]:
    ensure_docs_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        SELECT index_id
        FROM documents
        WHERE doc_name = ? AND cohort_name = ?
        """,
        (doc_name, cohort_name),
    )
    row = cur.fetchone()
    conn.close()
    if row:
        return row[0]
    return None


def list_all_documents() -> List[Tuple[str, str, str]]:
    """
    Return list of (doc_name, cohort_name, created_at).
    """
    ensure_docs_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        "SELECT doc_name, cohort_name, created_at FROM documents ORDER BY created_at DESC"
    )
    rows = cur.fetchall()
    conn.close()
    return rows
# ------------------------------------------------------------
# SCHEMA BOOTSTRAP (ensures required tables exist for tests)
# ------------------------------------------------------------
ensure_docs_table()
ensure_cohort_table()



STEP 4 starting with DB_PATH: /content/drive/MyDrive/rag_mvp/rag_documents.db


In [19]:
# ============================================================


# CELL 4.5 / STEP 4.5 – Users & Chat History (7-Day Retention)


In [20]:
# ============================================================


from dataclasses import dataclass
import datetime as dt



In [21]:
# ============================================================

# USER IDENTITY MODEL


In [22]:
# ============================================================


@dataclass
class SessionUser:
    username: str | None = None
    role: str = "anonymous"   # "anonymous", "user", "admin"

    @property
    def is_authenticated(self) -> bool:
        return bool(self.username and str(self.username).strip()) and self.role in ("user", "admin")

    @property
    def is_admin(self) -> bool:
        return self.role == "admin"


def require_login(session_user, action: str = "access this feature") -> SessionUser:
    """Enforce authenticated access (no access for anonymous users).

    Accepts SessionUser, dict, or None and returns a normalized SessionUser.
    Raises PermissionError if not authenticated.
    """
    if isinstance(session_user, SessionUser):
        su = session_user
    elif isinstance(session_user, dict):
        su = SessionUser(
            username=((session_user.get("username") or "").strip() or None),
            role=(session_user.get("role") or "anonymous"),
        )
    else:
        su = SessionUser(username=None, role="anonymous")

    if not su.is_authenticated:
        raise PermissionError(f"Login required to {action}.")
    return su

# MVP in-memory auth store (will be replaced later by ICAM/SSO)
USERS = {
    "admin": {"password": "admin123", "role": "admin"},
    "demo1":  {"password": "demo1123",  "role": "user"},
    "demo2": {"password": "demo2123", "role": "user"},
}




In [23]:
# ============================================================

# USERS TABLE


In [24]:
# ============================================================


def ensure_user_table():
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS users (
            user_id      TEXT PRIMARY KEY,
            display_name TEXT,
            role         TEXT,
            created_at   TEXT NOT NULL
        )
        """
    )
    conn.commit()
    conn.close()


def upsert_user(user_id: str, role: str, display_name: Optional[str] = None):
    """
    Basic ICAM-ready user record.
    Inserts new or updates existing users.
    """
    if not user_id:
        return

    ensure_user_table()
    conn = get_db_conn()
    cur = conn.cursor()
    now = dt.datetime.now(dt.timezone.utc).isoformat()

    cur.execute(
        """
        INSERT INTO users (user_id, display_name, role, created_at)
        VALUES (?, ?, ?, ?)
        ON CONFLICT(user_id) DO UPDATE SET
            display_name = COALESCE(?, users.display_name),
            role = COALESCE(?, users.role)
        """,
        (user_id, display_name, role, now, display_name, role),
    )
    conn.commit()
    conn.close()




In [25]:
# ============================================================

# CHAT HISTORY TABLE


In [26]:
# ============================================================


def ensure_chat_history_table():
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS chat_history (
            id             INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id        TEXT,
            role           TEXT,
            cohort_name    TEXT,
            original_query TEXT,
            improved_query TEXT,
            which_prompt   TEXT,
            answer         TEXT,
            chat_model     TEXT,
            created_at     TEXT NOT NULL
        )
        """
    )
    conn.commit()
    conn.close()




In [27]:
# ============================================================

# 7-DAY RETENTION


In [28]:
# ============================================================


def prune_chat_history(days: int = 7):
    ensure_chat_history_table()
    cutoff = dt.datetime.now(dt.timezone.utc) - dt.timedelta(days=days)
    cutoff_iso = cutoff.isoformat()

    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute("DELETE FROM chat_history WHERE created_at < ?", (cutoff_iso,))
    conn.commit()
    conn.close()




In [29]:
# ============================================================

# v14 BEHAVIOR: Save Chat Interaction (DETAILED LOGGING)


In [30]:
# ============================================================


def save_chat_interaction(
    user_id: str,
    role: str,
    cohort_name: str,
    original_query: str,
    improved_query: str,
    which_prompt: str,
    answer: str,
    chat_model: str,
):
    """
    This is your existing v16 logging mechanism.
    Now fully preserved and compatible with v16.
    """
    ensure_chat_history_table()
    prune_chat_history(days=7)

    conn = get_db_conn()
    cur = conn.cursor()
    now = dt.datetime.now(dt.timezone.utc).isoformat()

    cur.execute(
        """
        INSERT INTO chat_history (
            user_id, role, cohort_name, original_query, improved_query,
            which_prompt, answer, chat_model, created_at
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
        (
            user_id or None,
            role or None,
            cohort_name or None,
            original_query,
            improved_query,
            which_prompt,
            answer,
            chat_model,
            now,
        ),
    )
    conn.commit()
    conn.close()




In [31]:
# ============================================================

# v16 REQUIRED FUNCTIONS – FLEXIBLE SIGNATURES


In [32]:
# ============================================================


def _extract_user_id(user: Any) -> str:
    """
    Helper to derive a stable user_id from various representations.
    """
    if user is None:
        return "anonymous"

    # dict-like
    if isinstance(user, dict):
        for key in ("username", "user_id", "name"):
            v = user.get(key)
            if v:
                return str(v)

    # attribute-based (SessionUser or other objects)
    for attr in ("username", "user_id", "name"):
        try:
            v = getattr(user, attr, None)
            if v:
                return str(v)
        except Exception:
            pass

    # mapping-like (sqlite3.Row, etc.)
    try:
        for key in ("username", "user_id", "name"):
            if key in user:
                v = user[key]
                if v:
                    return str(v)
    except Exception:
        pass

    return "anonymous"


def save_chat_history(
    messages=None,
    *args,
    user: Any = None,
    user_id: Optional[str] = None,
    cohort_name: str = "default",
    cohort: Optional[str] = None,
    **kwargs,
):
    """
    v16 Test Suite Requirement:
    Supports BOTH:
      1) save_chat_history(messages=[{role, content}, ...], user=..., cohort=...)
      2) save_chat_history(user=..., cohort=..., question="Q", answer="A", model_used="...")

    In (2) we write a single row using the Q/A fields.
    """
    ensure_chat_history_table()
    prune_chat_history(days=7)

    # Normalize cohort alias
    if cohort is not None:
        cohort_name = cohort

    # Derive user_id if needed
    if user_id is None:
        user_id = _extract_user_id(user)

    # ---- Path 1: test-style call with question/answer/model_used ----
    question = kwargs.get("question")
    answer = kwargs.get("answer")
    model_used = kwargs.get("model_used")

    if messages is None and (question is not None or answer is not None):
        now = dt.datetime.now(dt.timezone.utc).isoformat()
        conn = get_db_conn()
        cur = conn.cursor()

        cur.execute(
            """
            INSERT INTO chat_history (
                user_id, role, cohort_name, original_query,
                improved_query, which_prompt, answer,
                chat_model, created_at
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                user_id,          # user_id
                "user",           # role
                cohort_name,      # cohort_name
                question or "",   # original_query
                None,             # improved_query
                None,             # which_prompt
                answer or "",     # answer
                model_used,       # chat_model
                now,              # created_at
            ),
        )

        conn.commit()
        conn.close()
        return  # We're done for this style of call

    # ---- Path 2: normal messages-based usage ----
    if messages is None:
        messages = kwargs.get("messages", None)

    # If still no messages, treat as no-op
    if messages is None:
        return

    now = dt.datetime.now(dt.timezone.utc).isoformat()
    conn = get_db_conn()
    cur = conn.cursor()

    for m in messages:
        role = m.get("role", "user")
        content = m.get("content", "")

        cur.execute(
            """
            INSERT INTO chat_history (
                user_id, role, cohort_name, original_query,
                improved_query, which_prompt, answer,
                chat_model, created_at
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                user_id,
                role,
                cohort_name,
                content,   # original_query
                None,
                None,
                None,
                None,
                now,
            ),
        )

    conn.commit()
    conn.close()



def load_chat_history(
    *args,
    **kwargs,
):
    """
    v16 Test Suite Requirement:
    Returns list[dict] with keys: role, content, created_at.

    For simplicity and maximum compatibility with the tests,
    we ignore user/cohort filters here and just return the
    oldest messages up to `limit`.
    """
    ensure_chat_history_table()

    limit = int(kwargs.get("limit", 50))

    conn = get_db_conn()
    cur = conn.cursor()

    cur.execute(
        """
        SELECT role, original_query, created_at
        FROM chat_history
        ORDER BY created_at ASC
        LIMIT ?
        """,
        (limit,),
    )

    rows = cur.fetchall()
    conn.close()

    return [
        {"role": r[0], "content": r[1], "created_at": r[2]}
        for r in rows
    ]





In [33]:
# ============================================================

# RECENT HISTORY (Admin screens / Debug UI)


In [34]:
# ============================================================


def get_recent_history(
    user_id: Optional[str] = None,
    cohort_name: Optional[str] = None,
    limit: int = 50,
) -> List[Dict[str, Any]]:
    """
    Returns full detailed history rows for admin/debug views.
    """
    ensure_chat_history_table()

    conn = get_db_conn()
    cur = conn.cursor()

    query = """
        SELECT user_id, role, cohort_name, original_query, improved_query,
               which_prompt, answer, chat_model, created_at
        FROM chat_history
    """
    params = []
    conditions = []

    if user_id:
        conditions.append("user_id = ?")
        params.append(user_id)
    if cohort_name:
        conditions.append("cohort_name = ?")
        params.append(cohort_name)

    if conditions:
        query += " WHERE " + " AND ".join(conditions)

    query += " ORDER BY created_at DESC LIMIT ?"
    params.append(limit)

    cur.execute(query, params)
    rows = cur.fetchall()
    conn.close()

    return [
        {
            "user_id": r[0],
            "role": r[1],
            "cohort_name": r[2],
            "original_query": r[3],
            "improved_query": r[4],
            "which_prompt": r[5],
            "answer": r[6],
            "chat_model": r[7],
            "created_at": r[8],
        }
        for r in rows
    ]




In [35]:
# ============================================================

# ADMIN: LIST USERS


In [36]:
# ============================================================


def list_users() -> List[Tuple[str, str, str]]:
    ensure_user_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute("SELECT user_id, display_name, role FROM users ORDER BY created_at DESC")
    rows = cur.fetchall()
    conn.close()
    return rows

# CELL 4.6 / STEP 4.6 – Audit Log (v16)


In [37]:
# ============================================================


def ensure_audit_table():
    """
    Create an audit_log table if it doesn't already exist.
    """
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS audit_log (
            id        INTEGER PRIMARY KEY AUTOINCREMENT,
            ts        TEXT NOT NULL,
            username  TEXT,
            role      TEXT,
            action    TEXT NOT NULL,
            details   TEXT
        )
        """
    )
    conn.commit()
    conn.close()


def log_audit(username: str, role: str, action: str, details: str = ""):
    """
    Insert a row into the audit_log table.
    - ts: UTC ISO timestamp
    - username / role: may be None/empty for anonymous
    - action: short code, e.g. 'login', 'ask', 'admin_refresh', 'delete_cohort'
    - details: freeform string with context
    """
    ensure_audit_table()
    conn = get_db_conn()
    cur = conn.cursor()

    # Use the global datetime alias `dt` so we don't conflict with any
    # later "import datetime" inside the self-test cell.
    now = dt.datetime.now(dt.timezone.utc).isoformat()

    cur.execute(
        """
        INSERT INTO audit_log (ts, username, role, action, details)
        VALUES (?, ?, ?, ?, ?)
        """,
        (now, username or "", role or "", action, details or ""),
    )
    conn.commit()
    conn.close()


def trace_log(message: str):
    """
    Append a timestamped debug line to debug_trace_v16.log in BASE_DIR.

    Used to capture errors/warnings that happen inside Gradio callbacks
    where the UI might just show a generic 'Error'.
    """
    try:
        ts = dt.datetime.now(dt.timezone.utc).isoformat()
    except Exception:
        ts = "UNKNOWN_TIME"

    line = f"{ts} {message}\n"
    try:
        with open(TRACE_LOG_PATH, "a", encoding="utf-8") as f:
            f.write(line)
    except Exception as e:
        # Last resort: don't let logging itself crash anything
        print("TRACE_LOG_ERROR", e, line)



import traceback as _tb

def safe_ui_call(fn, fallback=None):
    """Wrap a Gradio callback so exceptions don't surface as generic 'Error' UI outputs.
    Writes full traceback to trace_log() and returns a safe fallback value.
    """
    if fallback is None:
        fallback = "❌ An internal error occurred. See debug trace log."
    def _wrapped(*args, **kwargs):
        try:
            return fn(*args, **kwargs)
        except Exception as e:
            try:
                trace_log(f"[UI ERROR] {getattr(fn, '__name__', 'callback')}: {e}\n{_tb.format_exc()}")
            except Exception:
                pass
            return fallback
    return _wrapped


In [38]:
# ============================================================

# STEP 4.7 – Cohort Ownership & Sharing (v16-safe)


In [39]:
# ============================================================


from typing import Optional, List, Dict, Any

def ensure_cohort_meta_table():
    """
    Metadata for cohorts (v16_1 Phase 1):
      - owner_user_id: who created/owns the cohort (REQUIRED)
      - is_shared: 0 = private to owner/admin, 1 = visible to all authenticated users
      - allow_clone: 0 = non-owners cannot clone, 1 = cloning enabled for authenticated users (when shared)
      - created_ts: UTC timestamp

    Notes:
      - Performs a lightweight schema migration if cohort_meta already exists.
    """
    conn = get_db_conn()
    cur = conn.cursor()

    # Create base table if missing
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS cohort_meta (
            cohort_name    TEXT PRIMARY KEY,
            owner_user_id  TEXT,
            is_shared      INTEGER DEFAULT 0,
            created_ts     TEXT
        )
        """
    )

    # ---- Schema migration (add allow_clone if missing) ----
    try:
        cur.execute("PRAGMA table_info(cohort_meta)")
        cols = {r[1] for r in cur.fetchall()}  # r[1] = column name
        if "allow_clone" not in cols:
            cur.execute("ALTER TABLE cohort_meta ADD COLUMN allow_clone INTEGER DEFAULT 0")
    except Exception as e:
        # Do not hard-fail cohort operations due to migration issues
        trace_log(f"ensure_cohort_meta_table migration warning: {e}")

    conn.commit()
    conn.close()

def _extract_username_from_user(user: SessionUser | dict | None) -> Optional[str]:
    """
    Helper: accept SessionUser, dict, or None and pull out a username string.
    """
    if user is None:
        return None

    # If SessionUser dataclass
    if isinstance(user, SessionUser):
        return user.username

    # If dict-like
    if isinstance(user, dict):
        # Try common keys in order
        for k in ("username", "user_id", "name"):
            try:
                v = user.get(k)
                if v:
                    return str(v)
            except Exception:
                pass

    return None


def set_cohort_owner(cohort_name: str, user: SessionUser | dict | None):
    """
    Register the owner of a cohort, but do NOT change it once set.

    HARD REQUIREMENT (v16_1 Phase 1 security):
      - Cohorts must have an authenticated owner.
      - We do NOT silently fall back to "anonymous".

    Behavior:
      - On first creation: insert owner_user_id and is_shared=0 (private).
      - On subsequent calls for the same cohort_name: owner remains unchanged.
    """
    if not cohort_name:
        return

    from datetime import datetime, timezone

    owner = _extract_username_from_user(user)
    if not owner or not str(owner).strip() or str(owner).strip().lower() == "anonymous":
        raise ValueError("Cohort owner is required. Please log in before creating/updating a cohort.")


    ensure_cohort_meta_table()
    conn = get_db_conn()
    cur = conn.cursor()

    now = datetime.now(timezone.utc).isoformat()

    # Insert cohort_meta row if missing; if it already exists with an invalid owner, repair the owner once.
    cur.execute(
        """
        INSERT INTO cohort_meta (cohort_name, owner_user_id, is_shared, created_ts)
        VALUES (?, ?, 0, ?)
        ON CONFLICT(cohort_name) DO UPDATE SET
            owner_user_id = excluded.owner_user_id
        WHERE cohort_meta.owner_user_id IS NULL
           OR TRIM(cohort_meta.owner_user_id) = ''
           OR LOWER(TRIM(cohort_meta.owner_user_id)) = 'anonymous'
        """,
        (cohort_name, owner, now),
    )


    conn.commit()
    conn.close()


def set_cohort_sharing(cohort_name: str, is_shared: bool | int):
    """
    Update ONLY the is_shared flag for a cohort in cohort_meta.

    is_shared:
        False/0 -> private (owner + admins)
        True/1  -> shared (all users can see; non-owners are read-only)
    """
    if not cohort_name:
        return

    ensure_cohort_meta_table()
    conn = get_db_conn()
    cur = conn.cursor()
    shared_flag = 1 if is_shared else 0

    cur.execute(
        """
        UPDATE cohort_meta
        SET is_shared = ?
        WHERE cohort_name = ?
        """,
        (shared_flag, cohort_name),
    )
    conn.commit()
    conn.close()

def set_cohort_clone_allowed(cohort_name: str, allow_clone: bool | int):
    """
    Update ONLY the allow_clone flag for a cohort in cohort_meta.

    allow_clone:
        False/0 -> non-owners cannot clone (even if shared)
        True/1  -> if shared, any authenticated user may clone
    """
    if not cohort_name:
        return

    ensure_cohort_meta_table()
    conn = get_db_conn()
    cur = conn.cursor()
    flag = 1 if allow_clone else 0

    cur.execute(
        """
        UPDATE cohort_meta
        SET allow_clone = ?
        WHERE cohort_name = ?
        """,
        (flag, cohort_name),
    )
    conn.commit()
    conn.close()

def set_cohort_allow_clone(cohort_name: str, allow_clone: bool) -> None:
    """
    Backward-compatible alias for older call sites.
    """
    return set_cohort_clone_allowed(cohort_name, allow_clone)


#def set_cohort_clone_allowed(cohort_name: str, allow_clone: bool | int):
#    """Backward-compatible alias for earlier UI code."""
#    return set_cohort_clone_allowed(cohort_name, allow_clone)

def _get_cohort_meta(cohort_name: str) -> tuple[str | None, int, int]:
    """
    Returns (owner_user_id, is_shared, allow_clone).
    If cohort not found in meta, returns (None, 0, 0).
    """
    ensure_cohort_meta_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        "SELECT owner_user_id, COALESCE(is_shared,0), COALESCE(allow_clone,0) FROM cohort_meta WHERE cohort_name = ?",
        (cohort_name,),
    )
    row = cur.fetchone()
    conn.close()
    if not row:
        return None, 0, 0
    return (row[0], int(row[1] or 0), int(row[2] or 0))


def can_clone_cohort(session_user: Any, source_cohort: str) -> tuple[bool, str | None]:
    """
    Returns (allowed, message_if_denied).

    Rules (Option A):
      - Admin: may always clone.
      - Owner: may always clone.
      - Other authenticated users:
          * allowed ONLY if source cohort is_shared=1 AND allow_clone=1
      - Anonymous: not allowed.
    """
    if not source_cohort:
        return False, "❌ Please select a cohort to clone."

    username = _extract_user_id(session_user)
    if not username or username == "anonymous":
        return False, "❌ You must be logged in to clone a cohort."

    # Determine admin role
    role = None
    if isinstance(session_user, dict):
        role = session_user.get("role")
    elif isinstance(session_user, SessionUser):
        role = "admin" if session_user.is_admin else "user"
    else:
        role = USERS.get(username, {}).get("role", "user")

    if role == "admin":
        return True, None

    owner, is_shared, allow_clone = _get_cohort_meta(source_cohort)

    # Owner may always clone
    if owner and owner == username:
        return True, None

    # Non-owner: only if shared + cloning allowed
    if is_shared == 1 and allow_clone == 1:
        return True, None

    return False, "❌ Cloning is not permitted for this cohort (owner has disabled cloning)."


def clone_cohort(source_cohort: str, target_cohort: str, new_owner: str) -> str:
    """
    Clone a cohort by duplicating its document rows and index artifacts.

    Implementation notes:
      - We copy each source FAISS index (.faiss) and metadata (.pkl) to a new index_id.
      - We create new rows in documents and cohort_docs for the target cohort.
      - The cloned cohort is created as PRIVATE by default (is_shared=0, allow_clone=0).
    """
    if not source_cohort or not target_cohort:
        return "❌ Source and target cohort names are required."

    if not new_owner or not str(new_owner).strip() or str(new_owner).strip().lower() == "anonymous":
        return "❌ You must be logged in to clone a cohort (owner is required)."


    target_cohort = target_cohort.strip()
    if cohort_exists(target_cohort):
        return f"❌ Target cohort '{target_cohort}' already exists. Choose a different name."

    ensure_docs_table()
    ensure_cohort_table()
    ensure_cohort_meta_table()

    conn = get_db_conn()
    cur = conn.cursor()

    cur.execute(
        "SELECT doc_name, index_id, n_chunks, embed_model FROM documents WHERE cohort_name = ?",
        (source_cohort,),
    )
    rows = cur.fetchall()

    if not rows:
        conn.close()
        return f"❌ Source cohort '{source_cohort}' has no documents to clone."

    created_at = dt.datetime.now(dt.timezone.utc).isoformat()

    # Create a meta row for the new cohort (private by default)
    cur.execute(
        """
        INSERT INTO cohort_meta (cohort_name, owner_user_id, is_shared, allow_clone, created_ts)
        VALUES (?, ?, 0, 0, ?)
        ON CONFLICT(cohort_name) DO NOTHING
        """,
        (target_cohort, (new_owner or "").strip(), created_at),
    )

    cloned_count = 0
    for (doc_name, src_index_id, n_chunks, embed_model) in rows:
        new_index_id = str(uuid4())

        # Copy FAISS + PKL artifacts to new ids
        src_faiss = os.path.join(INDEX_DIR, f"{src_index_id}.faiss")
        src_pkl = os.path.join(INDEX_DIR, f"{src_index_id}.pkl")
        dst_faiss = os.path.join(INDEX_DIR, f"{new_index_id}.faiss")
        dst_pkl = os.path.join(INDEX_DIR, f"{new_index_id}.pkl")

        if not os.path.exists(src_faiss) or not os.path.exists(src_pkl):
            trace_log(f"clone_cohort warning: missing artifacts for index_id={src_index_id}")
            continue

        shutil.copyfile(src_faiss, dst_faiss)
        shutil.copyfile(src_pkl, dst_pkl)

        # Insert new document row
        new_doc_id = str(uuid4())
        cur.execute(
            """
            INSERT INTO documents (id, doc_name, cohort_name, index_id, n_chunks, embed_model, created_at)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """,
            (new_doc_id, doc_name, target_cohort, new_index_id, int(n_chunks), embed_model, created_at),
        )

        # Map doc to cohort_docs
        cur.execute(
            "INSERT INTO cohort_docs (cohort_name, doc_name) VALUES (?, ?)",
            (target_cohort, doc_name),
        )

        cloned_count += 1

    conn.commit()
    conn.close()

    return f"✅ Cloned cohort '{source_cohort}' into '{target_cohort}' with {cloned_count} documents."


def cohort_exists(cohort_name: str) -> bool:
    """
    Returns True if a cohort with this name already exists in cohort_docs.
    This is global (not per-user) to avoid confusing duplicate names.
    """
    if not cohort_name:
        return False

    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        "SELECT 1 FROM cohort_docs WHERE cohort_name = ? LIMIT 1",
        (cohort_name,),
    )
    row = cur.fetchone()
    conn.close()
    return row is not None


def list_cohorts_for_user(user: SessionUser | dict | None) -> list[str]:
    """
    Return list of cohort names visible to the given user.

    Rules:
      - Admins: all cohorts (global)
      - Non-admin:
          * Cohorts where they are owner (cohort_meta.owner_user_id)
          * Cohorts marked is_shared = 1
      - Anonymous: only shared cohorts (is_shared = 1)

    If anything goes wrong with the metadata logic, falls back to global list_cohorts().
    Also writes debug info to the v16 trace log.
    """
    trace_log(f"list_cohorts_for_user called with user={user!r}")

    ensure_cohort_table()
    ensure_cohort_meta_table()

    conn = get_db_conn()
    cur = conn.cursor()

    try:
        # Determine if user is admin, safely
        is_admin = False
        username = None

        if isinstance(user, SessionUser):
            is_admin = user.is_admin
            username = user.username
        elif isinstance(user, dict):
            username = _extract_username_from_user(user)
            role = user.get("role")
            is_admin = (role == "admin")

        if is_admin:
            # Admin sees all distinct cohorts
            cur.execute(
                """
                SELECT DISTINCT cohort_name
                FROM cohort_docs
                ORDER BY cohort_name
                """
            )
        else:
            # Non-admin or anonymous
            if username:
                # Logged-in non-admin -> owner or shared
                cur.execute(
                    """
                    SELECT DISTINCT cd.cohort_name
                    FROM cohort_docs cd
                    LEFT JOIN cohort_meta cm
                      ON cd.cohort_name = cm.cohort_name
                    WHERE cm.owner_user_id = ?
                       OR cm.is_shared = 1
                       OR cm.cohort_name IS NULL   -- safety: cohorts without meta still appear
                    ORDER BY cd.cohort_name
                    """,
                    (username,),
                )
            else:
                # Anonymous -> only shared (or cohorts w/o meta as a fallback)
                cur.execute(
                    """
                    SELECT DISTINCT cd.cohort_name
                    FROM cohort_docs cd
                    LEFT JOIN cohort_meta cm
                      ON cd.cohort_name = cm.cohort_name
                    WHERE cm.is_shared = 1
                       OR cm.cohort_name IS NULL
                    ORDER BY cd.cohort_name
                    """
                )

        rows = cur.fetchall()
        conn.close()
        names = [r[0] for r in rows]

        # Final safety net: do not leak global cohorts; return empty.
        if not names:
            trace_log("list_cohorts_for_user -> no rows; returning empty")
            names = []

        trace_log(f"list_cohorts_for_user returning {names}")
        return names

    except Exception as e:
        conn.close()
        trace_log(f"list_cohorts_for_user ERROR: {e}")
        return []



# ------------------------------------------------------------
# UI HELPERS FOR COHORT LISTS (used in STEP 10)
# ------------------------------------------------------------

def get_cohorts_for_user(username: Optional[str]) -> List[str]:
    """
    Convenience wrapper for the Gradio UI:
    take a simple username string and delegate to list_cohorts_for_user().

    Also logs to the v16 trace file for easier debugging of Refresh Cohorts.
    """
    try:
        trace_log(f"get_cohorts_for_user called with username={username!r}")

        if not username or not str(username).strip():
            # No anonymous access: callers must be logged in.
            return []
        role = USERS.get(username, {}).get("role", "user")
        user = SessionUser(username=str(username).strip(), role=role)

        names = list_cohorts_for_user(user)
        trace_log(f"get_cohorts_for_user returning {names}")
        return names

    except Exception as e:
        trace_log(f"get_cohorts_for_user ERROR for username={username!r}: {e}")
        # No anonymous fallback; return empty to avoid leaking data.
        return []



def get_all_cohorts() -> List[List[str]]:
    """
    For the Admin tab: return [[cohort_name, owner], ...].
    """
    ensure_cohort_table()
    ensure_cohort_meta_table()

    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        SELECT cd.cohort_name,
               COALESCE(cm.owner_user_id, 'unknown') AS owner
        FROM cohort_docs cd
        LEFT JOIN cohort_meta cm
          ON cd.cohort_name = cm.cohort_name
        GROUP BY cd.cohort_name
        ORDER BY cd.cohort_name
        """
    )
    rows = cur.fetchall()
    conn.close()

    return [[name, owner] for (name, owner) in rows]

def get_allowed_setup_actions(current_user_dict: dict | None) -> tuple[list[str], str, str, bool]:
    """
    Returns:
      - action_choices
      - action_value (safe default)
      - info_message
      - info_visible
    """
    # Build a SessionUser safely
    if not current_user_dict or not current_user_dict.get("username"):
        user_obj = SessionUser(username=None, role="anonymous")
    else:
        user_obj = SessionUser(
            username=current_user_dict["username"],
            role=current_user_dict.get("role", "user"),
        )

    # Determine if clone is possible for this user
    cloneable = list_cloneable_shared_cohorts_for_user(user_obj)  # you already have / can add this helper

    choices = [ACTION_CREATE, ACTION_APPEND]
    msg = ""
    visible = False

    if cloneable:
        choices.append(ACTION_CLONE)
    else:
        # “Greyed out” effect: omit the choice entirely
        msg = (
            "Clone is unavailable because no shared cohorts have been marked as clone-eligible for your account.\n\n"
            "Ask the cohort owner (or an admin) to enable **Allow cloning** on a shared cohort."
        )
        visible = True

    # Always return a valid selection (prevents the ValueError you hit on logout)
    return choices, DEFAULT_ACTION, msg, visible






In [40]:
# ============================================================

# CELL 4.7 / STEP 4.7 – Demo Prompts (v16.1)


In [41]:
# ============================================================

# Supports "demo mode" cohorts by storing curated prompt questions in SQLite.
# Prompts can be preloaded via admin file upload (CSV/TXT/JSON).

def ensure_demo_prompts_table():
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS demo_prompts (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            cohort_name TEXT NOT NULL,
            prompt_text TEXT NOT NULL,
            sort_order INTEGER DEFAULT 0,
            created_at TEXT DEFAULT (datetime('now'))
        )
        """
    )
    cur.execute("CREATE INDEX IF NOT EXISTS idx_demo_prompts_cohort ON demo_prompts(cohort_name)")
    conn.commit()
    conn.close()

def clear_demo_prompts_for_cohort(cohort_name: str):
    ensure_demo_prompts_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute("DELETE FROM demo_prompts WHERE cohort_name = ?", (cohort_name,))
    conn.commit()
    conn.close()

def list_demo_prompts_for_cohort(cohort_name: str) -> list[str]:
    ensure_demo_prompts_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        SELECT prompt_text
        FROM demo_prompts
        WHERE cohort_name = ?
        ORDER BY COALESCE(sort_order,0), id
        """,
        (cohort_name,),
    )
    rows = cur.fetchall()
    conn.close()
    return [r[0] for r in rows if r and r[0]]

def upsert_demo_prompts_for_cohort(cohort_name: str, prompts: list[str], replace: bool = True) -> int:
    """
    Insert demo prompts for a cohort. If replace=True, clears existing prompts first.
    Returns number of prompts inserted.
    """
    ensure_demo_prompts_table()
    cohort = (cohort_name or "").strip()
    if not cohort:
        return 0

    cleaned: list[str] = []
    seen: set[str] = set()
    for p in (prompts or []):
        t = (p or "").strip()
        if not t:
            continue
        if t in seen:
            continue
        seen.add(t)
        cleaned.append(t)

    if replace:
        clear_demo_prompts_for_cohort(cohort)

    if not cleaned:
        return 0

    conn = get_db_conn()
    cur = conn.cursor()
    for i, t in enumerate(cleaned, start=1):
        cur.execute(
            "INSERT INTO demo_prompts (cohort_name, prompt_text, sort_order) VALUES (?,?,?)",
            (cohort, t, i),
        )
    conn.commit()
    conn.close()
    return len(cleaned)

def parse_demo_prompts_file(file_path: str) -> list[str]:
    """
    Parse demo prompts from a local file path.
    Supported:
      - .txt: one question per line (blank lines ignored)
      - .csv: one prompt per line OR header with columns 'prompt'/'question'/'text'
      - .json: either list[str] or {"prompts":[...]}
    """
    if not file_path:
        return []
    fp = str(file_path)
    ext = os.path.splitext(fp)[1].lower()
    try:
        with open(fp, "r", encoding="utf-8", errors="ignore") as f:
            raw = f.read()
    except Exception:
        return []

    if ext == ".json":
        try:
            obj = json.loads(raw)
            if isinstance(obj, list):
                return [str(x) for x in obj]
            if isinstance(obj, dict) and isinstance(obj.get("prompts"), list):
                return [str(x) for x in obj.get("prompts")]
        except Exception:
            return []
        return []

    lines = [ln.strip() for ln in raw.replace("\r\n", "\n").replace("\r", "\n").split("\n")]
    lines = [ln for ln in lines if ln]

    if ext == ".txt":
        return lines

    if ext == ".csv":
        if not lines:
            return []
        header = [h.strip().strip('"').strip("'").lower() for h in lines[0].split(",")]
        if any(h in ("prompt", "question", "text") for h in header):
            col_idx = None
            for k in ("prompt", "question", "text"):
                if k in header:
                    col_idx = header.index(k)
                    break
            out = []
            for row in lines[1:]:
                parts = [p.strip().strip('"').strip("'") for p in row.split(",")]
                if col_idx is not None and col_idx < len(parts):
                    out.append(parts[col_idx])
            return out
        # no header: each row is a prompt (first column)
        out = []
        for row in lines:
            parts = [p.strip().strip('"').strip("'") for p in row.split(",")]
            if parts and parts[0]:
                out.append(parts[0])
        return out

    return lines



In [42]:
# ============================================================


# CELL 5.0 / STEP 5.0 – Model Registry Core (v16_1, with schema migration)


In [43]:
# ============================================================


import sqlite3
from typing import List, Dict, Any

def ensure_model_registry_table():
    """
    Ensure the model_registry table exists with the v6_15 schema.
    If an older schema is detected (missing model_id or other key fields),
    we DROP and recreate the table.

    This is safe for the MVP since we don't rely on persistent custom models yet.
    """
    conn = get_db_conn()
    cur = conn.cursor()

    # Does the table exist?
    cur.execute(
        "SELECT name FROM sqlite_master WHERE type='table' AND name='model_registry'"
    )
    row = cur.fetchone()

    if row:
        # Table exists – inspect its columns
        cur.execute("PRAGMA table_info(model_registry)")
        cols_info = cur.fetchall()
        existing_cols = {c[1] for c in cols_info}  # c[1] is the column name

        required_cols = {
            "provider",
            "model_id",
            "display_name",
            "model_type",
            "enabled",
            "is_default",
            "cost_score",
            "latency_score",
            "max_context_tokens",
            "api_base",
            "notes",
        }

        # If the key v16 columns are missing, drop and recreate
        if not required_cols.issubset(existing_cols):
            print("DEBUG: model_registry schema mismatch detected. Dropping old table.")
            cur.execute("DROP TABLE IF EXISTS model_registry")
            conn.commit()

    # (Re)create the table with the correct v16 schema
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS model_registry (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            provider TEXT,
            model_id TEXT,
            display_name TEXT,
            model_type TEXT,
            enabled INTEGER DEFAULT 1,
            is_default INTEGER DEFAULT 0,
            cost_score INTEGER DEFAULT 2,
            latency_score INTEGER DEFAULT 2,
            max_context_tokens INTEGER,
            api_base TEXT,
            notes TEXT
        )
        """
    )
    conn.commit()
    conn.close()


def seed_default_models():
    """
    Seed the registry with standard defaults if empty.
    """
    ensure_model_registry_table()
    conn = get_db_conn()
    cur = conn.cursor()

    cur.execute("SELECT COUNT(*) FROM model_registry")
    count = cur.fetchone()[0]

    if count == 0:
        defaults = [
            # Default Chat Model
            {
                "provider": "openai",
                "model_id": "gpt-4.1-mini",
                "display_name": "GPT-4.1 Mini",
                "model_type": "chat",
                "enabled": 1,
                "is_default": 1,
                "cost_score": 1,
                "latency_score": 1,
                "max_context_tokens": 128_000,
                "api_base": None,
                "notes": "Primary chat model",
            },
            # Default Embedding Model
            {
                "provider": "openai",
                "model_id": "text-embedding-3-small",
                "display_name": "text-embedding-3-small",
                "model_type": "embed",
                "enabled": 1,
                "is_default": 1,
                "cost_score": 1,
                "latency_score": 1,
                "max_context_tokens": None,
                "api_base": None,
                "notes": "Primary embedding model",
            },
        ]

        for row in defaults:
            cur.execute(
                """
                INSERT INTO model_registry (
                    provider, model_id, display_name, model_type,
                    enabled, is_default, cost_score, latency_score,
                    max_context_tokens, api_base, notes
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                (
                    row["provider"],
                    row["model_id"],
                    row["display_name"],
                    row["model_type"],
                    row["enabled"],
                    row["is_default"],
                    row["cost_score"],
                    row["latency_score"],
                    row["max_context_tokens"],
                    row["api_base"],
                    row["notes"],
                ),
            )

        conn.commit()

    conn.close()


def list_models(model_type: str | None = None, only_enabled: bool = True) -> List[Dict[str, Any]]:
    """
    Return raw dict rows from the model_registry table.

    Args:
        model_type: 'chat', 'embed', etc. (optional)
        only_enabled: filter to enabled == 1
    """
    ensure_model_registry_table()
    conn = get_db_conn()
    cur = conn.cursor()

    sql = """
        SELECT provider, model_id, display_name, model_type,
               enabled, is_default, cost_score, latency_score,
               max_context_tokens, api_base, notes
        FROM model_registry
        WHERE 1=1
    """
    params: list[Any] = []

    if model_type:
        sql += " AND model_type = ?"
        params.append(model_type)

    if only_enabled:
        sql += " AND enabled = 1"

    sql += " ORDER BY model_type, is_default DESC, model_id"

    cur.execute(sql, params)
    rows = cur.fetchall()
    conn.close()

    keys = [
        "provider", "model_id", "display_name", "model_type", "enabled",
        "is_default", "cost_score", "latency_score", "max_context_tokens",
        "api_base", "notes",
    ]

    return [dict(zip(keys, r)) for r in rows]
def load_model_registry() -> List[Dict[str, Any]]:
    """
    Loads all models from the registry and ensures defaults exist.
    """
    ensure_model_registry_table()
    seed_default_models()  # <-- CRITICAL LINE (missing previously)

    conn = get_db_conn()
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()

    cur.execute(
        """
        SELECT
            id,
            provider,
            model_id,
            model_id AS model,          -- backward compatibility
            display_name,
            model_type AS type,         -- backward compatibility
            enabled,
            is_default,
            cost_score,
            latency_score,
            max_context_tokens,
            api_base,
            notes
        FROM model_registry
        ORDER BY model_type, is_default DESC, model_id
        """
    )

    rows = [dict(r) for r in cur.fetchall()]
    conn.close()
    return rows



In [44]:
# ============================================================


# CELL 5.1 / STEP 5.1 – ModelConfig & Lookup Helpers (v16)


In [45]:
# ============================================================


from dataclasses import dataclass
from typing import Optional, List, Dict, Any

@dataclass
class ModelConfig:
    model_id: str
    provider: str = "openai"
    display_name: str | None = None
    model_type: str = "chat"   # 'chat', 'embed', 'rerank', etc.
    is_default: bool = False
    enabled: bool = True
    cost_score: int = 2
    latency_score: int = 2
    max_context_tokens: int | None = None
    api_base: str | None = None
    notes: str | None = None

    def as_kwargs(self) -> dict:
        """
        Common kwargs we might pass into a client, e.g. OpenAI.
        For now this is mostly for future extensibility.
        """
        kw = {"model": self.model_id}
        if self.api_base:
            kw["api_base"] = self.api_base
        return kw


def load_model_configs(model_type: str, only_enabled: bool = True) -> list[ModelConfig]:
    """
    Load models from the registry as ModelConfig objects.
    Relies on list_models(...) from STEP 5.0.
    """
    rows = list_models(model_type=model_type, only_enabled=only_enabled)
    configs: list[ModelConfig] = []
    for row in rows:
        configs.append(
            ModelConfig(
                model_id=row["model_id"],
                provider=row["provider"],
                display_name=row["display_name"],
                model_type=row["model_type"],
                is_default=row["is_default"],
                enabled=row["enabled"],
                cost_score=row["cost_score"],
                latency_score=row["latency_score"],
                max_context_tokens=row["max_context_tokens"],
                api_base=row["api_base"],
                notes=row["notes"],
            )
        )
    return configs


def get_default_model(model_type: str) -> Optional[Dict[str, Any]]:
    """
    Low-level helper used by the *Config() helpers below.

    Looks in the model_registry for the default model of a given type.
    Strategy:
      1) Use list_models(model_type, only_enabled=True)
      2) Return the one where is_default = True, if present
      3) Otherwise return the first enabled model
      4) If none exist, return None
    """
    rows = list_models(model_type=model_type, only_enabled=True)
    if not rows:
        return None

    for row in rows:
        if row.get("is_default"):
            return row

    # Fallback: first enabled of that type
    return rows[0]


def get_default_chat_model_config() -> ModelConfig:
    """
    Returns a ModelConfig for the default chat model.
    If none is explicitly set, falls back to the first enabled chat model.
    """
    default_row = get_default_model("chat")
    if default_row:
        return ModelConfig(
            model_id=default_row["model_id"],
            provider=default_row["provider"],
            display_name=default_row["display_name"],
            model_type=default_row["model_type"],
            is_default=default_row["is_default"],
            enabled=default_row["enabled"],
            cost_score=default_row["cost_score"],
            latency_score=default_row["latency_score"],
            max_context_tokens=default_row["max_context_tokens"],
            api_base=default_row["api_base"],
            notes=default_row["notes"],
        )

    # Fallback: first enabled chat model
    configs = load_model_configs("chat", only_enabled=True)
    if configs:
        return configs[0]

    # Last resort hard-coded default (should not happen due to seeding)
    return ModelConfig(
        model_id="gpt-4.1-mini",
        provider="openai",
        display_name="GPT-4.1 Mini (fallback)",
        model_type="chat",
        is_default=True,
        enabled=True,
        cost_score=1,
        latency_score=1,
        max_context_tokens=128_000,
    )


def get_default_embed_model_config() -> ModelConfig:
    """
    Returns a ModelConfig for the default embedding model.
    """
    default_row = get_default_model("embed")
    if default_row:
        return ModelConfig(
            model_id=default_row["model_id"],
            provider=default_row["provider"],
            display_name=default_row["display_name"],
            model_type=default_row["model_type"],
            is_default=default_row["is_default"],
            enabled=default_row["enabled"],
            cost_score=default_row["cost_score"],
            latency_score=default_row["latency_score"],
            max_context_tokens=default_row["max_context_tokens"],
            api_base=default_row["api_base"],
            notes=default_row["notes"],
        )

    # Fallback: first enabled embed model
    configs = load_model_configs("embed", only_enabled=True)
    if configs:
        return configs[0]

    # Last resort hard-coded default
    return ModelConfig(
        model_id="text-embedding-3-small",
        provider="openai",
        display_name="text-embedding-3-small (fallback)",
        model_type="embed",
        is_default=True,
        enabled=True,
        cost_score=1,
        latency_score=1,
    )
def list_chat_models() -> List[str]:
    models = load_model_registry()
    return [m["model_id"] for m in models if m["type"] == "chat" and m["enabled"]]



In [46]:
# ============================================================


# CELL 5.2 / STEP 5.2 – RAG Retrieval Over a Cohort (v16)


In [47]:
# ============================================================


def build_context_from_index(
    api_key: str,
    chat_model: str,
    embed_model: str,
    cohort_name: str,
    query: str,
    top_k: int = 5,
):
    """
    Given a cohort and query:
    - Load all documents for that cohort
    - For each doc's index, perform similarity search
    - Aggregate top_k results across docs
    Returns:
      - concatenated context string
      - list of (doc_name, rank, score) for citations
    """

    ensure_docs_table()
    resolved_chat, resolved_embed = resolve_models(chat_model, embed_model)
    client = build_openai_client(api_key)

    # 1. Load all docs & their index references
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        SELECT doc_name, index_id
        FROM documents
        WHERE cohort_name = ?
        """,
        (cohort_name,),
    )
    rows = cur.fetchall()
    conn.close()

    if not rows:
        return "", []

    # 2. Embed query once
    q_embed_resp = client.embeddings.create(model=resolved_embed, input=[query])
    q_vec = np.array(q_embed_resp.data[0].embedding, dtype="float32")
    q_vec = q_vec / (np.linalg.norm(q_vec) + 1e-10)

    all_hits = []  # (doc_name, idx, score, text)

    # 3. Perform similarity search in each doc index
    for doc_name, index_id in rows:
        try:
            index = load_index(index_id)
            meta = load_metadata(index_id)  # {"chunks": [...]}
        except Exception:
            continue

        D, I = index.search(q_vec[np.newaxis, :], top_k)
        scores = D[0]
        idxs = I[0]

        for score, idx in zip(scores, idxs):
            if idx < 0:
                continue
            chunks = meta.get("chunks", [])
            if idx >= len(chunks):
                continue
            text_chunk = chunks[idx]
            all_hits.append((doc_name, idx, float(score), text_chunk))

    if not all_hits:
        return "", []

    # 4. Sort & select top results
    all_hits.sort(key=lambda x: x[2], reverse=True)
    top_hits = all_hits[:top_k]

    context_parts = []
    citations = []

    for rank, (doc_name, idx, score, text) in enumerate(top_hits, start=1):
        header = f"[{rank}] From {doc_name} (chunk #{idx}, score={score:.3f})"
        context_parts.append(header + "\n" + text)
        citations.append((doc_name, rank, score))

    context = "\n\n".join(context_parts)
    return context, citations




In [48]:
# ============================================================

# UPDATED answer_with_rag() — Now uses Routing Brain (v16)


In [49]:
# ============================================================


def answer_with_rag(
    api_key: str,
    chat_model: str,          # kept for backward compatibility
    embed_model: str,         # still used for embedding queries
    cohort_name: str,
    query: str,
    system_prompt: str = "",
    user_pref: str | None = None,  # NEW: allows override from UI
):
    """
    Perform RAG retrieval and return:
        - answer_markdown
        - raw_answer_text
        - model_used (for history & UI display)
    """

    # 1. Build RAG context
    context, citations = build_context_from_index(
        api_key, chat_model, embed_model, cohort_name, query
    )

    if not context:
        return (
            "I could not find any context for this query in the selected cohort.",
            "",
            "N/A",
        )

    # 2. Default system prompt
    if not system_prompt:
        system_prompt = (
            "You are a helpful assistant answering questions based on the provided context.\n"
            "If the answer cannot be found in the context, say you do not know."
        )

    # 3. Construct chat messages
    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": (
                "Use ONLY the context below to answer the question.\n\n"
                "=== CONTEXT START ===\n"
                f"{context}\n"
                "=== CONTEXT END ===\n\n"
                f"QUESTION: {query}"
            ),
        },
    ]

    # 4. Call the routing brain — NEW for v16
    answer_text, raw_answer_text, model_used = call_chat_model(
        api_key=api_key,
        messages=messages,
        task_type="rag_answer",
        user_pref=user_pref,          # user-selected override (optional)
        context_size=len(context),    # helps routing choose large/small models
    )

    # 5. Build answer markdown with citations + model info
    md = answer_text + "\n\n---\n\n**Cited sources:**\n"
    for doc_name, rank, score in citations:
        md += f"- [{rank}] `{doc_name}` (score={score:.3f})\n"

    md += f"\n\n**Model Used:** `{model_used}`\n"

    return md, raw_answer_text, model_used
def answer_question_over_cohort(api_key, username, cohort_name, question, model_id):
    """
    Wrapper for the RAG retrieval + LLM step.
    Must exist BEFORE STEP 10.
    """
    trace_log(
        f"answer_question_over_cohort called user={username}, cohort={cohort_name}"
    )

    embed_cfg = get_default_embed_model_config()
    embed_model = embed_cfg.model_id

    answer_md, raw_answer, used_model = answer_with_rag(
        api_key=api_key,
        chat_model=model_id,
        embed_model=embed_model,
        cohort_name=cohort_name,
        query=question,
        system_prompt="",
        user_pref=model_id,  # routed through routing brain
    )

    return answer_md, raw_answer, used_model



In [50]:
# ============================================================


# CELL 6 / STEP 6 – Build Cohort from Uploaded Docs (v16)


In [51]:
# ============================================================

def build_cohort_from_files(
    api_key: str,
    embed_model: str,
    cohort_name: str,
    files: List[Any],
) -> str:
    """Ingest a list of uploaded files into a *single* cohort.

    For each file we:
      - load the text
      - chunk it
      - embed with the given embedding model
      - build & save a FAISS index
      - save metadata (including the chunks)
      - register the document in the SQLite `documents` table

    Returns a human-readable summary string for the UI.
    """
    ensure_docs_table()
    ensure_cohort_table()
    os.makedirs(INDEX_DIR, exist_ok=True)

    success_count = 0
    total_chunks = 0
    messages: List[str] = []

    for file_obj in files:
        try:
            # 1) Load text & derive a stable doc_name
            text, doc_name = load_file_to_text(file_obj)
            if not text or not text.strip():
                messages.append(f"⚠️ {doc_name}: no extractable text, skipped.")
                continue

            # 2) Chunk
            chunks = chunk_text(text)
            if not chunks:
                messages.append(f"⚠️ {doc_name}: produced 0 chunks, skipped.")
                continue

            # 3) Embed
            vectors = embed_texts(
                api_key=api_key,
                embed_model=embed_model,
                texts=chunks,
            )

            # 4) Build FAISS index
            index = build_faiss_index(vectors)

            # 5) Save index & metadata
            index_id = str(uuid4())
            save_index(index, index_id)

            meta = {
                "cohort_name": cohort_name,
                "doc_name": doc_name,
                "chunks": chunks,
                "embed_model": embed_model,
            }
            save_metadata(index_id, meta)

            # 6) Register in SQLite
            register_document(
                doc_name=doc_name,
                cohort_name=cohort_name,
                index_id=index_id,
                n_chunks=len(chunks),
                embed_model=embed_model,
            )

            success_count += 1
            total_chunks += len(chunks)
            messages.append(f"✅ {doc_name}: {len(chunks)} chunks embedded.")

        except Exception as e:
            # Best-effort error capture per-file
            name = getattr(file_obj, "name", str(file_obj))
            messages.append(f"❌ {name}: {e}")

    if success_count == 0:
        detail = "\n".join(messages) if messages else ""
        return "❌ No documents were successfully processed." + (f"\n{detail}" if detail else "")

    summary = (
        f"✅ Built cohort '{cohort_name}' with {success_count} document(s) "
        f"and {total_chunks} total chunks."
    )
    if messages:
        summary += "\n" + "\n".join(messages)
    return summary


def build_cohort_index(
    api_key: str,
    cohort_name: str,
    files: List[Any],
    owner: Optional[str] = None,
) -> str:
    """
    High-level helper used by the Gradio UI (v16):

      - Selects default embedding model (from model registry)
      - Builds the cohort
      - Stores cohort ownership
    """
    if not api_key or not api_key.strip():
        return "❌ OpenAI API key is required."

    if not cohort_name or not cohort_name.strip():
        return "❌ Cohort name is required."

    if (not owner) or (not str(owner).strip()) or (str(owner).strip().lower() in ("anonymous","none","null")):
        return "❌ You must be logged in to create a cohort (owner is required)."

    if not files:
        return "❌ Please upload at least one file."

    # ✔️ NEW — correct default embedding model call
    embed_cfg = get_default_embed_model_config()
    embed_model = embed_cfg.model_id

    # Build FAISS index + metadata
    result_msg = build_cohort_from_files(
        api_key=api_key,
        embed_model=embed_model,
        cohort_name=cohort_name.strip(),
        files=files,
    )

    # Save cohort owner metadata (required)


    # Save cohort owner metadata (required)
    try:
        role = USERS.get(owner, {}).get("role", "user")
        user_obj = SessionUser(username=str(owner).strip(), role=role)
        set_cohort_owner(cohort_name.strip(), user_obj)
    except Exception as e:
        trace_log(f"build_cohort_index set_cohort_owner ERROR: {e}")
        return f"❌ Failed to set cohort owner (cohort will not be treated as valid): {e}"

    return result_msg




In [52]:
# ============================================================


# CELL 6.1 / STEP 6.1 – Routing Brain (v16)


In [53]:
# ============================================================

#
# Centralized model selection + wrapper for ALL chat LLM calls.
# Uses the model_registry table (STEP 5.0) and supports:
#   - task_type hints ("question_improve", "rag_answer", "summary", "admin")
#   - optional user_pref override (a specific model_id)
#   - dry_run flag for automated self-tests (no API call made)
#
# Functions:
#   - select_chat_model(task_type, context_size, user_pref)
#   - call_chat_model(api_key, messages, task_type, user_pref, context_size, dry_run)

from typing import List, Dict, Any, Tuple


def select_chat_model(
    task_type: str,
    context_size: int = 0,
    user_pref: str | None = None,
) -> str:
    """
    Decide which chat model_id to use based on:
      - user_pref: explicit override from UI
      - registry defaults (is_default)
      - task_type and context_size (reserved for future heuristics)

    Returns:
        model_id (e.g., "gpt-4.1-mini")
    """
    # Load enabled chat models as ModelConfig objects
    configs = load_model_configs("chat", only_enabled=True)

    # Fallback if registry is empty or misconfigured
    if not configs:
        return "gpt-4.1-mini"

    # 1. If user_pref matches a known enabled model_id, honor it
    if user_pref:
        for cfg in configs:
            if cfg.model_id == user_pref:
                return cfg.model_id

    # 2. Prefer the model marked as default
    for cfg in configs:
        if cfg.is_default:
            return cfg.model_id

    # 3. Simple heuristic placeholder:
    #    For now we ignore task_type/context_size and just use the first enabled.
    return configs[0].model_id



def call_chat_model(
    api_key: str,
    messages: List[Dict[str, Any]],
    task_type: str,
    user_pref: str | None = None,
    context_size: int = 0,
    dry_run: bool = False,
) -> Tuple[str, str, str]:
    """
    Wrapper for ALL chat LLM calls in the app.

    Args:
        api_key:      OpenAI API key
        messages:     Chat completion messages
        task_type:    Semantic label for routing (e.g. 'rag_answer', 'question_improve')
        user_pref:    Optional explicit model_id override
        context_size: Approx size of context (chars) to inform routing
        dry_run:      If True, DO NOT call the API (used by self-tests).

    Returns:
        (answer_text, raw_answer_text, model_used)
    """
    model_id = select_chat_model(
        task_type=task_type,
        context_size=context_size,
        user_pref=user_pref,
    )

    # For automated self-tests: don't hit the API
    if dry_run:
        dummy = f"[DRY RUN] task_type={task_type}, model_id={model_id}"
        return dummy, dummy, model_id

    client = build_openai_client(api_key)

    resp = client.chat.completions.create(
        model=model_id,
        messages=messages,
        temperature=0.2,
        max_tokens=900,
    )

    answer_text = resp.choices[0].message.content.strip()
    # In this MVP, raw_answer_text == answer_text, but we keep both for future transforms
    raw_answer_text = answer_text

    return answer_text, raw_answer_text, model_id



In [54]:
# ============================================================


# CELL 7 / STEP 7 – Admin Helpers (Stats & Maintenance)


In [55]:
# ============================================================


def get_db_stats() -> str:
    ensure_docs_table()
    ensure_cohort_table()
    ensure_user_table()
    ensure_chat_history_table()

    conn = get_db_conn()
    cur = conn.cursor()

    cur.execute("SELECT COUNT(*) FROM documents")
    n_docs = cur.fetchone()[0]

    cur.execute("SELECT COUNT(DISTINCT cohort_name) FROM cohort_docs")
    n_cohorts = cur.fetchone()[0]

    cur.execute("SELECT COUNT(*) FROM users")
    n_users = cur.fetchone()[0]

    cur.execute("SELECT COUNT(*) FROM chat_history")
    n_chats = cur.fetchone()[0]

    conn.close()

    return (
        f"**DB Stats**\n\n"
        f"- Documents: {n_docs}\n"
        f"- Cohorts: {n_cohorts}\n"
        f"- Users: {n_users}\n"
        f"- Chat records (last 7 days enforced on write): {n_chats}\n"
    )

def describe_users() -> str:
    rows = list_users()
    if not rows:
        return "No users have been recorded yet."
    lines = ["**Known Users**\n"]
    for user_id, display_name, role in rows:
        disp = display_name or "(no display name)"
        r = role or "(no role)"
        lines.append(f"- `{user_id}` – {disp} – role: `{r}`")
    return "\n".join(lines)

def describe_cohorts() -> str:
    """
    Returns a markdown summary of cohorts, including:
      - name
      - owner
      - visibility (private/shared)
      - document count
    """
    ensure_docs_table()
    ensure_cohort_table()
    ensure_cohort_meta_table()

    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        SELECT DISTINCT
            cd.cohort_name,
            COALESCE(cm.owner_user_id, '(none)') AS owner,
            COALESCE(cm.is_shared, 0)            AS is_shared,
            COUNT(DISTINCT cd.doc_name)          AS num_docs
        FROM cohort_docs cd
        LEFT JOIN cohort_meta cm
          ON cd.cohort_name = cm.cohort_name
        GROUP BY cd.cohort_name, owner, is_shared
        ORDER BY cd.cohort_name
        """
    )
    rows = cur.fetchall()
    conn.close()

    if not rows:
        return "No cohorts found."

    lines = ["**Cohorts**", ""]
    for name, owner, is_shared, num_docs in rows:
        share_label = "shared" if is_shared else "private"
        lines.append(
            f"- **{name}** — owner: `{owner}`, visibility: {share_label}, docs: {num_docs}"
        )

    return "\n".join(lines)



In [56]:
# ============================================================


# CELL 8 / STEP 8 – Prompt Coach (Optional Query Improvement) – v16


In [57]:
# ============================================================

#
# Uses the v16 Routing Brain (call_chat_model) instead of calling OpenAI directly.
# Signature is kept the same so STEP 10's on_improve_query(...) still works:
#     improve_query(api_key, chat_model, original_query)
#
# In a future step, we can optionally add a user-selected model override and
# pass it into call_chat_model(user_pref=...).

def improve_query(
    api_key: str,
    chat_model: str,       # kept for backward compatibility; routing ignores it
    original_query: str,
) -> str:
    """
    Prompt coach to re-write the user's query for better RAG retrieval.

    Behavior:
    - If the original query is empty/whitespace, returns "".
    - Otherwise, uses the Routing Brain (task_type='question_improve') to pick
      an appropriate chat model and rewrite the question to be clearer, more
      explicit, and RAG-friendly.
    """
    if not original_query.strip():
        return ""

    system_prompt = (
        "You are a prompt coach helping the user improve questions for a RAG system. "
        "Rewrite the query to be explicit, concise, and focused on key details. "
        "Preserve the user's intent but remove ambiguity, vague pronouns, and "
        "unnecessary filler. Return ONLY the improved query text."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": original_query},
    ]

    # Use the v16 Routing Brain instead of calling OpenAI directly
    improved, _, model_used = call_chat_model(
        api_key=api_key,
        messages=messages,
        task_type="question_improve",
        user_pref=None,                     # (optional override will come from UI later)
        context_size=len(original_query),   # small, but available for routing heuristics
    )

    # For now we just return the improved text. If desired later, we can:
    # - log model_used to audit_log
    # - display which model did the improvement in the UI.
    return improved.strip()



In [58]:
# ============================================================


# CELL 9 / STEP 9 – Chat History Viewer (User-Facing)


In [59]:
# ============================================================


def format_history_markdown(
    user_id: Optional[str],
    cohort_name: Optional[str],
    limit: int = 50,
) -> str:
    """
    Turn recent history into markdown for display.
    """
    hist = get_recent_history(user_id=user_id, cohort_name=cohort_name, limit=limit)
    if not hist:
        return "No chat history found for the given filters (within retention window)."

    lines = []
    lines.append(
        f"**Showing up to {limit} most recent interactions** "
        f"{'(filtered)' if user_id or cohort_name else ''}\n"
    )

    for h in hist:
        ts = h["created_at"]
        u = h["user_id"] or "(anonymous)"
        r = h["role"] or "(none)"
        c = h["cohort_name"] or "(none)"
        which = h["which_prompt"] or "(unknown)"

        lines.append(f"---\n**User:** `{u}`  |  **Role:** `{r}`  |  **Cohort:** `{c}`  |  **When:** {ts}")
        lines.append(f"**Prompt used:** `{which}`")
        lines.append(f"**Original query:**\n{h['original_query']}\n")
        if h["improved_query"]:
            lines.append(f"**Improved query:**\n{h['improved_query']}\n")
        lines.append("**Answer:**")
        lines.append(h["answer"])
        lines.append("")

    return "\n".join(lines)



In [ ]:
# ============================================================


#============================================================
# NOTE: If RUN_PREFLIGHT_ONLY=True, do not launch Gradio
#============================================================
if globals().get("STOP_AFTER_PREFLIGHT", False):
    print("✅ Pre-flight only: skipping Gradio launch.")
else:
    # CELL 9.5 / STEP 9.5 – Identity & Admin Ops (v14)
    # ============================================================
    def authenticate(username: str, password: str) -> SessionUser | None:
        """
        MVP auth: checks against local USERS dict.
        Returns SessionUser or None if invalid.
        """
        record = USERS.get(username)
        if not record:
            return None
        if password != record["password"]:
            return None
        return SessionUser(username=username, role=record["role"])


    def authenticate_credentials(username: str, password: str):
        """
        Wrapper used by the Gradio login logic in STEP 10.

        It calls authenticate(...) which returns a SessionUser, then:
          - Upserts the user into the `users` table (for admin/history views)
          - Returns a simple dict {username, role} that the UI expects.
        """

        # Use the existing MVP auth
        user = authenticate(username, password)
        if not user:
            return None

        # Make sure the user exists in the DB's `users` table
        try:
            upsert_user(
                user_id=user.username,
                role=user.role,
                display_name=user.username,
            )
        except Exception as e:
            # Don't break login if DB write fails; just log it
            trace_log(f"authenticate_credentials upsert_user ERROR: {e}")

        # UI login code in STEP 10 expects a dict-like object
        return {"username": user.username, "role": user.role}
        # Use the existing MVP auth
        user = authenticate(username, password)
        if not user:
            return None

        # Make sure the user exists in the DB's `users` table
        try:
            upsert_user(
                user_id=user.username,
                role=user.role,
                display_name=user.username,
            )
        except Exception as e:
            # Don't break login if DB write fails; just log it
            trace_log(f"authenticate_credentials upsert_user ERROR: {e}")

        # UI login code in STEP 10 expects a dict-like object
        return {"username": user.username, "role": user.role}

    def require_admin(user: SessionUser):
        """
        Helper for admin-only actions. Raises PermissionError if not admin.
        """
        if not user or not user.is_admin:
            raise PermissionError("Admin privileges required for this action.")


    def admin_delete_cohort(user: SessionUser, cohort_name: str) -> str:
        """
        Admin-only wrapper around delete_cohort().
        Uses the existing delete_cohort function from STEP 4.
        """
        try:
            require_admin(user)
        except PermissionError as e:
            return f"❌ Not authorized: {e}"

        if not cohort_name:
            return "❌ Please select a cohort to delete."

        try:
            # Use existing v13 delete_cohort logic (no reassignment in this MVP).
            msg = delete_cohort(cohort_name, reassign_to=None)
            log_audit(user.username, user.role, "delete_cohort", f"cohort={cohort_name}")
            return msg
        except Exception as e:
            return f"❌ Error deleting cohort: {e}"

    def admin_view_audit_log(user: SessionUser, limit: int = 50) -> str:
        """
        Admin-only view of recent audit log entries.
        """
        try:
            require_admin(user)
        except PermissionError as e:
            return f"❌ Not authorized: {e}"

        ensure_audit_table()
        conn = get_db_conn()
        cur = conn.cursor()
        cur.execute(
            """
            SELECT ts, username, role, action, details
            FROM audit_log
            ORDER BY id DESC
            LIMIT ?
            """,
            (limit,),
        )
        rows = cur.fetchall()
        conn.close()

        if not rows:
            return "No audit log entries."

        lines = ["**Recent Audit Log Entries**\n"]
        for ts, username, role, action, details in rows:
            u = username or "-"
            r = role or "-"
            d = details or ""
            lines.append(f"- {ts} | user=`{u}` | role=`{r}` | action=`{action}` | {d}")

        return "\n".join(lines)

    # ============================================================
    # STEP 10 — Gradio App, Tabs & Startup (v16_1 with cohort sharing)
    # ============================================================

    # -----------------------------
    # Setup & Cohorts Tab
    # -----------------------------

    def build_setup_tab(session_state, api_key_state):
        """
        Setup tab (v16_1 Phase 1):
          - Create a new cohort from uploaded files (authenticated users only)
          - Append files to an existing cohort (owner/admin only)
          - Clone a shared cohort when (is_shared=1 AND allow_clone=1)

        IMPORTANT POLICY:
          - No access is allowed unless the user is logged in.
          - Cohorts must always have a valid owner; 'anonymous' is never permitted.
        """
        gr.Markdown("### Build or Manage a Cohort")

        def _require_session_user(session_user, action: str) -> SessionUser:
            # require_login() is defined earlier in the codebase; this wrapper keeps UI callbacks tidy.
            return require_login(session_user, action)

        def _is_admin(su: SessionUser) -> bool:
            return bool(su and su.is_admin)

        def _get_owner(cohort_name: str) -> str | None:
            ensure_cohort_meta_table()
            conn = get_db_conn()
            cur = conn.cursor()
            cur.execute("SELECT owner_user_id FROM cohort_meta WHERE cohort_name = ? LIMIT 1", (cohort_name,))
            row = cur.fetchone()
            conn.close()
            return row[0] if row else None

        def _can_write(session_user, cohort_name: str) -> bool:
            try:
                su = _require_session_user(session_user, "modify cohorts")
            except PermissionError:
                return False
            if _is_admin(su):
                return True
            owner = _get_owner(cohort_name)
            return bool(owner and su.username and owner == su.username)

        def _list_cloneable_shared_cohorts() -> list[str]:
            """
            Cloneable cohorts = is_shared=1 AND allow_clone=1.
            We join through `documents` to ensure the cohort has content.
            """
            ensure_cohort_meta_table()
            ensure_docs_table()
            conn = get_db_conn()
            cur = conn.cursor()
            cur.execute(
                """
                SELECT DISTINCT cm.cohort_name
                FROM cohort_meta cm
                JOIN documents d
                  ON d.cohort_name = cm.cohort_name
                WHERE cm.is_shared = 1
                  AND COALESCE(cm.allow_clone,0) = 1
                  AND d.cohort_name IS NOT NULL
                  AND TRIM(d.cohort_name) <> ''
                ORDER BY cm.cohort_name
                """
            )
            rows = cur.fetchall()
            conn.close()
            return [r[0] for r in rows]

        # ---------- UI ----------
        with gr.Row():
            action_radio = gr.Radio(
                choices=[ACTION_CREATE, ACTION_APPEND, ACTION_CLONE],
                value=DEFAULT_ACTION,
                label="Action",
            )

        clone_info_md = gr.Markdown("", visible=False)

        with gr.Row():
            new_cohort_name = gr.Textbox(label="New Cohort Name", placeholder="e.g., WIC_Security_2025")
            existing_cohort_dropdown = gr.Dropdown(
                label="Existing Cohort",
                choices=[],
                interactive=True,
                visible=False,
            )

        with gr.Row():
            share_checkbox = gr.Checkbox(label="Shared (visible to all logged-in users)", value=False)
            allow_clone_checkbox = gr.Checkbox(label="Allow other users to clone this cohort", value=False)

        file_uploader = gr.File(
            label="Upload Documents (PDF, DOCX, TXT)",
            file_count="multiple",
            type="filepath",
        )

        with gr.Row():
            build_btn = gr.Button("Run", variant="primary")

        gr.Markdown("#### Clone a Shared Cohort")
        with gr.Row():
            clone_source_dropdown = gr.Dropdown(
                label="Source Cohort (shared + cloning enabled)",
                choices=[],
                interactive=True,
                visible=False,
            )
            clone_target_name = gr.Textbox(
                label="New Cohort Name (clone target)",
                placeholder="e.g., My_Copy_of_WIC_Security_2025",
                visible=False,
            )

        with gr.Row():
            clone_btn = gr.Button("Clone Cohort", visible=False)

        build_status = gr.Markdown("")

        # ---------- dynamic visibility ----------
        def _toggle_fields(action: str, session_user):
            # ✅ FIX 1: Handle None/empty action by defaulting to CREATE
            if not action or action not in [ACTION_CREATE, ACTION_APPEND,       ACTION_CLONE]:
                action = ACTION_CREATE

            try:
                _require_session_user(session_user, "use cohort management")
            except PermissionError:
                # If not logged in, hide everything. (Login gating policy)
                return (
                    gr.update(visible=False),  # new_cohort_name
                    gr.update(visible=False, choices=[], value=None),  # existing_cohort_dropdown
                    gr.update(visible=False),  # share_checkbox
                    gr.update(visible=False),  # allow_clone_checkbox
                    gr.update(visible=False),  # file_uploader
                    gr.update(visible=False),  # build_btn
                    gr.update(visible=False, choices=[], value=None),  # clone_source_dropdown
                    gr.update(visible=False),  # clone_target_name
                    gr.update(visible=False),  # clone_btn
                    gr.update(value="❌ Please log in to manage cohorts.", visible=True),  # clone_info_md
                )

            is_create = action == ACTION_CREATE
            is_append = action == ACTION_APPEND
            is_clone = action == ACTION_CLONE

            clone_sources = _list_cloneable_shared_cohorts() if is_clone else []
            has_sources = len(clone_sources) > 0

            msg = ""
            if is_clone and not has_sources:
                msg = "ℹ️ No shared cohorts are available for you to clone yet."

            return (
                gr.update(visible=is_create),
                gr.update(visible=is_append),
                gr.update(visible=is_create or is_append),
                gr.update(visible=is_create or is_append),
                gr.update(visible=is_create or is_append),
                gr.update(visible=is_create or is_append),
                gr.update(visible=is_clone and has_sources, choices=clone_sources, value=None),
                gr.update(visible=is_clone and has_sources),
                gr.update(visible=is_clone and has_sources),
                gr.update(value=msg, visible=bool(msg)),
            )

        action_radio.change(
            fn=safe_ui_call(_toggle_fields, fallback=(
                gr.update(visible=True),   # new_cohort_name
                gr.update(visible=False),  # existing_cohort_dropdown
                gr.update(visible=True),   # share_checkbox
                gr.update(visible=True),   # allow_clone_checkbox
                gr.update(visible=True),   # file_uploader
                gr.update(visible=True),   # build_btn
                gr.update(visible=False, choices=[], value=None),  # clone_source_dropdown
                gr.update(visible=False, value=""),              # clone_target_name
                gr.update(visible=False),                         # clone_btn
                gr.update(value="❌ UI error. See trace log.", visible=True),  # clone_info_md
            )),
            inputs=[action_radio, session_state],
            outputs=[
                new_cohort_name,
                existing_cohort_dropdown,
                share_checkbox,
                allow_clone_checkbox,
                file_uploader,
                build_btn,
                clone_source_dropdown,
                clone_target_name,
                clone_btn,
                clone_info_md,
            ],
        )
        # ✅ FIX 2: REMOVE THIS ENTIRE BLOCK (it causes the bug by re-triggering with action=None)
        # Refresh the cohort-management UI when login/logout changes session_state.
        # This prevents stale "Please log in" messages from persisting after a successful login.
        session_state.change(
            fn=safe_ui_call(_toggle_fields, fallback=(
                gr.update(visible=True),   # new_cohort_name
                gr.update(visible=False),  # existing_cohort_dropdown
                gr.update(visible=True),   # share_checkbox
                gr.update(visible=True),   # allow_clone_checkbox
                gr.update(visible=True),   # file_uploader
                gr.update(visible=True),   # build_btn
                gr.update(visible=False, choices=[], value=None),  # clone_source_dropdown
                gr.update(visible=False, value=""),                # clone_target_name
                gr.update(visible=False),                          # clone_btn
                gr.update(value="", visible=False),                # clone_info_md  (CLEAR)
            )),
            inputs=[action_radio, session_state],
            outputs=[
                new_cohort_name,
                existing_cohort_dropdown,
                share_checkbox,
                allow_clone_checkbox,
                file_uploader,
                build_btn,
                clone_source_dropdown,
                clone_target_name,
                clone_btn,
                clone_info_md,
            ],
            queue=False,
        )


        # ---------- refresh helpers ----------


        def _refresh_existing_dropdown(session_user):
            try:
                su = _require_session_user(session_user, "view cohorts")
            except PermissionError:
                return gr.update(choices=[], value=None)
            return gr.update(choices=get_cohorts_for_user(su.username), value=None)

        def _refresh_clone_sources(session_user):
            try:
                _require_session_user(session_user, "view cloneable cohorts")
            except PermissionError:
                return gr.update(choices=[], value=None)
            choices = _list_cloneable_shared_cohorts()
            return gr.update(choices=choices, value=None)
        def _normalize_cohort_name(selected: str | None) -> str:
            s = (selected or "").strip()
            if not s:
                return ""
            # If dropdown shows "name/owner", keep only the cohort name
            return s.split("/", 1)[0].strip()

        # ---------- action handlers ----------
       # ---------- action handlers ----------
        def _run_action(action, api_key, session_user, new_name, existing_name, files, is_shared, allow_clone):
            """
            MUST return exactly 3 outputs (matches build_btn.click outputs):
              1) build_status (markdown)
              2) file_uploader (File component)
              3) existing_cohort_dropdown (Dropdown component)
            """

            # Helper: refresh existing cohort dropdown choices for this user
            def _refresh_existing_dd(owner_username: str):
                try:
                    choices = get_cohorts_for_user(owner_username)
                except Exception as e:
                    trace_log(f"SETUP refresh existing dropdown ERROR: {e}")
                    choices = []
                return gr.update(choices=choices, value=None)

            # 1) Require session user
            try:
                su = _require_session_user(session_user, "create or modify cohorts")
            except PermissionError as e:
                return f"❌ {e}", gr.update(), gr.update()

            owner = (su.username or "").strip()
            if not owner or owner.lower() == "anonymous":
                return "❌ You must be logged in to create or modify cohorts (owner is required).", gr.update(), gr.update()

            if not api_key or not str(api_key).strip():
                return "❌ Please provide an OpenAI API key.", gr.update(), gr.update()

            # -------------------------
            # CREATE
            # -------------------------
            if action == ACTION_CREATE:
                cohort_name = (new_name or "").strip()

                if not cohort_name:
                    return "❌ Please provide a new cohort name.", gr.update(), gr.update()
                if cohort_exists(cohort_name):
                    return f"❌ Cohort '{cohort_name}' already exists.", gr.update(), gr.update()
                if not files:
                    return "❌ Please upload at least one file.", gr.update(), gr.update()

                # Build index first
                try:
                    msg = build_cohort_index(api_key=api_key, cohort_name=cohort_name, files=files, owner=owner)
                except Exception as e:
                    trace_log(f"Create build_cohort_index ERROR: {e}")
                    return f"❌ Error building cohort: {e}", gr.update(), gr.update()

                # Persist metadata (REQUIRED)
                try:
                    ensure_cohort_meta_table()
                    set_cohort_owner(cohort_name, su)  # required
                    set_cohort_sharing(cohort_name, bool(is_shared))
                    set_cohort_allow_clone(cohort_name, bool(allow_clone))
                except Exception as e:
                    trace_log(f"Create cohort meta update ERROR: {e}")
                    return f"❌ Failed to persist cohort ownership/sharing metadata: {e}", gr.update(), gr.update()

                try:
                    log_audit(owner, su.role, "create_cohort", f"cohort={cohort_name}, shared={is_shared}, allow_clone={allow_clone}")
                except Exception as e:
                    trace_log(f"log_audit create_cohort ERROR: {e}")

                # SUCCESS: clear uploader + refresh dropdown
                refreshed_dd = _refresh_existing_dd(owner)
                return msg, gr.update(value=None), refreshed_dd

            # -------------------------
            # APPEND
            # -------------------------
            if action == ACTION_APPEND:
                cohort_name = _normalize_cohort_name(existing_name)

                if not cohort_name:
                    return "❌ Please select an existing cohort.", gr.update(), gr.update()
                if not cohort_exists(cohort_name):
                    return f"❌ Cohort '{cohort_name}' does not exist.", gr.update(), gr.update()
                if not files:
                    return "❌ Please upload at least one file.", gr.update(), gr.update()
                if not _can_write(session_user, cohort_name):
                    return "❌ Not authorized: only the cohort owner or an admin may append files.", gr.update(), gr.update()

                embed_cfg = get_default_embed_model_config()

                try:
                    msg = build_cohort_from_files(
                        api_key=api_key,
                        embed_model=embed_cfg.model_id,
                        cohort_name=cohort_name,
                        files=files
                    )
                except Exception as e:
                    trace_log(f"Append build_cohort_from_files ERROR: {e}")
                    return f"❌ Error appending files to cohort: {e}", gr.update(), gr.update()

                # Owner/admin may update sharing flags during append
                try:
                    ensure_cohort_meta_table()
                    set_cohort_owner(cohort_name, su)  # repairs invalid owners if needed
                    set_cohort_sharing(cohort_name, bool(is_shared))
                    set_cohort_allow_clone(cohort_name, bool(allow_clone))
                except Exception as e:
                    trace_log(f"Append cohort meta update ERROR: {e}")
                    return f"❌ Failed to update cohort metadata: {e}", gr.update(), gr.update()

                try:
                    log_audit(owner, su.role, "append_files", f"cohort={cohort_name}, shared={is_shared}, allow_clone={allow_clone}")
                except Exception as e:
                    trace_log(f"log_audit append_files ERROR: {e}")

                # SUCCESS: clear uploader + refresh dropdown
                refreshed_dd = _refresh_existing_dd(owner)
                return msg, gr.update(value=None), refreshed_dd

            # -------------------------
            # Unsupported
            # -------------------------
            return "❌ Unsupported action selected.", gr.update(), gr.update()

        build_btn.click(
            fn=safe_ui_call(_run_action, fallback=("❌ UI error. See trace log.", gr.update(), gr.update())),
            inputs=[
                action_radio,
                api_key_state,
                session_state,
                new_cohort_name,
                existing_cohort_dropdown,
                file_uploader,
                share_checkbox,
                allow_clone_checkbox,
            ],
            outputs=[build_status, file_uploader, existing_cohort_dropdown],

        )


        def _run_clone(api_key, session_user, source_cohort, target_cohort):
            try:
                su = _require_session_user(session_user, "clone cohorts")
            except PermissionError as e:
                return f"❌ {e}"

            if not api_key or not str(api_key).strip():
                return "❌ Please provide an OpenAI API key."
            if not source_cohort:
                return "❌ Please select a source cohort."
            if not target_cohort or not target_cohort.strip():
                return "❌ Please provide a new cohort name for the clone."

            allowed = set(_list_cloneable_shared_cohorts())
            if (source_cohort not in allowed) and not _is_admin(su):
                return "❌ Not authorized: cloning is not enabled for this cohort."

            msg = clone_cohort(source_cohort=source_cohort, target_cohort=target_cohort.strip(), new_owner=su.username)
            try:
                log_audit(su.username, su.role, "clone_cohort", f"{source_cohort} → {target_cohort.strip()}")
            except Exception as e:
                trace_log(f"log_audit clone_cohort ERROR: {e}")

            return msg

        clone_btn.click(
            fn=safe_ui_call(_run_clone),
            inputs=[api_key_state, session_state, clone_source_dropdown, clone_target_name],
            outputs=[build_status],
        )

        # Return handles expected by build_interface()
        return {
            "action_radio": action_radio,
            "new_cohort_name": new_cohort_name,
            "existing_cohort_dropdown": existing_cohort_dropdown,
            "share_checkbox": share_checkbox,
            "allow_clone_checkbox": allow_clone_checkbox,
            "file_uploader": file_uploader,
            "build_btn": build_btn,
            "clone_source_dropdown": clone_source_dropdown,
            "clone_target_name": clone_target_name,
            "clone_btn": clone_btn,
            "build_status": build_status,
            "clone_info_md": clone_info_md,
            "refresh_existing_dropdown": _refresh_existing_dropdown,
            "refresh_clone_sources": _refresh_clone_sources,
        }

    def build_ask_tab(session_state, api_key_state):
        """
        Ask tab:
          - Choose cohort + chat model
          - Optional prompt improvement
          - Ask question over cohort with RAG
          - View documents in selected cohort
        """
        gr.Markdown("### Ask Questions Over a Cohort")

        with gr.Row():
            cohort_dropdown = gr.Dropdown(
                label="Cohort",
                choices=[],
                elem_id="ask_cohort_dropdown",
            )
            model_dropdown = gr.Dropdown(
                label="Choose Chat Model",
                choices=list_chat_models(),
                elem_id="ask_model_dropdown",
            )

        with gr.Row():
            demo_prompt_dropdown = gr.Dropdown(
                label="Demo Prompt (optional)",
                choices=[],
                value=None,
                interactive=True,
                visible=False,
                elem_id="ask_demo_prompt_dropdown",
            )

        with gr.Row():
            question_box = gr.Textbox(
                label="Your Question",
                lines=3,
                elem_id="ask_question_box",
            )
            improved_box = gr.Textbox(
                label="Improved Prompt (optional, from Prompt Coach)",
                lines=3,
                elem_id="ask_improved_box",
            )



        with gr.Row():
            improve_btn = gr.Button("Improve Prompt", elem_id="ask_improve_btn")
            ask_btn = gr.Button("Ask Question", elem_id="ask_ask_btn")
            refresh_cohorts_btn = gr.Button("Refresh Cohorts", elem_id="ask_refresh_btn")


        with gr.Accordion("Documents in Selected Cohort", open=False):
            docs_md = gr.Markdown("_No cohort selected._")


        gr.Markdown("#### Answer")
        answer_md = gr.Markdown()

        # ---- Callbacks ----

        def _refresh_cohorts(session_user):
            """Refresh cohort list for the logged-in user only (no anonymous access)."""
            try:
                su = require_login(session_user, "view cohorts")
                names = get_cohorts_for_user(su.username)
                return gr.update(choices=names, value=None)
            except PermissionError:
                return gr.update(choices=[], value=None)
            except Exception as e:
                trace_log(f"ASK _refresh_cohorts ERROR: {e}")
                return gr.update(choices=[], value=None)



        def _refresh_demo_prompts_ui(cohort_name: str, session_user):
            """
            If selected cohort has demo prompts, show the Demo Prompt dropdown and populate choices.
            Always clears the question box on cohort change to avoid stale text between demos.
            """
            try:
                su = require_login(session_user, "view demo prompts")
            except PermissionError:
                return gr.update(visible=False, choices=[], value=None), gr.update(value="")

            c = (cohort_name or "").strip()
            if not c:
                return gr.update(visible=False, choices=[], value=None), gr.update(value="")

            prompts = list_demo_prompts_for_cohort(c)
            if not prompts:
                return gr.update(visible=False, choices=[], value=None), gr.update(value="")

            return gr.update(visible=True, choices=prompts, value=None), gr.update(value="")

        def _demo_prompt_pick_cb(selected_prompt: str):
            # Populate "Your Question" from the selected demo prompt.
            return gr.update(value=(selected_prompt or "").strip())
        def _improve_query_cb(original_query: str, api_key: str):
            if not api_key or not api_key.strip():
                return "❌ OpenAI API key is required."
            try:
                improved = improve_query(
                    api_key=api_key,
                    chat_model=CHAT_MODEL_DEFAULT,
                    original_query=original_query or "",
                )
                if not improved:
                    return original_query
                return improved
            except Exception as e:
                trace_log(f"ASK _improve_query_cb ERROR: {e}")
                return f"❌ Error improving query: {e}"

        def _ask_cb(
            cohort_name: str,
            question: str,
            improved_prompt: str,
            model_id: str,
            session_user,
            api_key: str,
        ):
            try:
                su = require_login(session_user, "ask questions")
            except PermissionError as e:
                return f"❌ {e}"

            username = su.username.strip()

            # Authorization: user must have visibility to cohort
            if cohort_name and cohort_name not in set(get_cohorts_for_user(username)):
                return "❌ Not authorized to access this cohort."

            if not api_key or not api_key.strip():
                return "❌ OpenAI API key is required."

            if not cohort_name:
                return "❌ Please choose a cohort."

            # Decide which prompt is used
            improved_clean = (improved_prompt or "").strip()
            question_clean = (question or "").strip()
            used_improved = bool(improved_clean)

            final_question = improved_clean if used_improved else question_clean
            if not final_question:
                return "❌ Please enter a question."

            try:
                answer_markdown, raw_answer, used_model = answer_question_over_cohort(
                    api_key=api_key,
                    username=username,
                    cohort_name=cohort_name,
                    question=final_question,
                    model_id=model_id or CHAT_MODEL_DEFAULT,
                )

                # Log to chat_history
                try:
                    save_chat_history(
                        user=session_user,
                        cohort=cohort_name,
                        question=final_question,
                        answer=raw_answer,
                        model_used=used_model,
                    )
                except Exception as log_e:
                    trace_log(f"ASK save_chat_history ERROR: {log_e}")

                # Add a visible note about which prompt was used
                if used_improved:
                    note = "**Note:** Used the *Improved Prompt* for this answer.\n\n"
                else:
                    note = "**Note:** Used your *Original Question* for this answer.\n\n"

                return note + (answer_markdown or "")

            except Exception as e:
                trace_log(f"ASK _ask_cb ERROR: {e}")
                return f"❌ Error while answering question: {e}"


        def _load_docs_for_cohort(session_user, cohort_name: str):
            """Show documents for the selected cohort (login required)."""
            try:
                su = require_login(session_user, "view cohort documents")
            except PermissionError:
                return "❌ Login required to view cohort documents."
            if not cohort_name:
                return "_No cohort selected._"
            # Authorization: user must have visibility to this cohort
            if cohort_name not in set(get_cohorts_for_user(su.username)):
                return "❌ Not authorized to view this cohort."
            try:
                conn = get_db_conn()
                cur = conn.cursor()
                cur.execute(
                    """
                    SELECT doc_name, created_at
                    FROM documents
                    WHERE cohort_name = ?
                    ORDER BY created_at
                    """,
                    (cohort_name,),
                )
                rows = cur.fetchall()
                conn.close()

                if not rows:
                    return f"_No documents found for cohort `{cohort_name}`._"

                lines = [f"**Documents in cohort `{cohort_name}`:**"]
                for doc_name, created_at in rows:
                    if created_at:
                        lines.append(f"- `{doc_name}` (added {created_at})")
                    else:
                        lines.append(f"- `{doc_name}`")
                return "\n".join(lines)
            except Exception as e:
                trace_log(f"ASK _load_docs_for_cohort ERROR: {e}")
                return f"❌ Error loading documents for cohort `{cohort_name}`."

        # Wire buttons
        refresh_cohorts_btn.click(
            _refresh_cohorts,
            inputs=[session_state],
            outputs=[cohort_dropdown],
        )

        improve_btn.click(
            _improve_query_cb,
            inputs=[question_box, api_key_state],
            outputs=[improved_box],
        )

        ask_btn.click(
            _ask_cb,
            inputs=[
                cohort_dropdown,
                question_box,
                improved_box,
                model_dropdown,
                session_state,
                api_key_state,
            ],
            outputs=[answer_md],
        )

        cohort_dropdown.change(
            _load_docs_for_cohort,
            inputs=[session_state, cohort_dropdown],
            outputs=[docs_md],
        )

        cohort_dropdown.change(
            _refresh_demo_prompts_ui,
            inputs=[cohort_dropdown, session_state],
            outputs=[demo_prompt_dropdown, question_box],
        )

        demo_prompt_dropdown.change(
            _demo_prompt_pick_cb,
            inputs=[demo_prompt_dropdown],
            outputs=[question_box],
        )

        return {
            "cohort_dropdown": cohort_dropdown,
            "model_dropdown": model_dropdown,
            "question_box": question_box,
            "improved_box": improved_box,
            "answer_markdown": answer_md,
            "docs_markdown": docs_md,
        }


    # -----------------------------
    # History Tab
    # -----------------------------
    def build_history_tab(session_state):
        """
        Simple history viewer using format_history_markdown.
        """
        gr.Markdown("### Chat History (read-only)")

        with gr.Row():
            history_user_filter = gr.Textbox(
                label="Filter by User ID (optional)",
                placeholder="Leave blank for all users",
            )
            history_cohort_filter = gr.Textbox(
                label="Filter by Cohort (optional)",
                placeholder="Leave blank for all cohorts",
            )

        with gr.Row():
            use_current_user_btn = gr.Button("Use Current User")
            refresh_history_btn = gr.Button("Refresh History")

        history_md = gr.Markdown()

        def _use_current_user(session_user):
            return _extract_user_id(session_user) or ""

        def _refresh_history(session_user, user_id: str, cohort_name: str):
            try:
                su = require_login(session_user, "view chat history")
            except PermissionError as e:
                return f"❌ {e}"

            is_admin = su.is_admin

            u_raw = (user_id or "").strip()
            c_raw = (cohort_name or "").strip()

            # Normalize common "All" sentinel values to None
            if u_raw.lower() in ("all users", "all", "*"):
                u_raw = ""
            if c_raw.lower() in ("all cohorts", "all", "*"):
                c_raw = ""

            u = u_raw or None
            c = c_raw or None

            # Non-admins can only view their own history regardless of filter
            if not is_admin:
                u = su.username

            try:
                return format_history_markdown(u, c, limit=50)
            except Exception as e:
                trace_log(f"HISTORY _refresh_history ERROR: {e}")
                return f"❌ Error loading history: {e}"


        use_current_user_btn.click(
            _use_current_user,
            inputs=[session_state],
            outputs=[history_user_filter],
        )

        refresh_history_btn.click(
            _refresh_history,
            inputs=[session_state, history_user_filter, history_cohort_filter],
            outputs=[history_md],
        )


        return {
            "history_user_filter": history_user_filter,
            "history_cohort_filter": history_cohort_filter,
            "history_markdown": history_md,
        }


    # -----------------------------
    # Admin Tab
    # -----------------------------
    def build_admin_tab(session_state):
        """
        Admin tab showing DB stats, users, cohorts, and audit log.
        Fix 3.3: prevent cross-button "stacking" by updating all 4 panes each click.
        """
        gr.Markdown("### Admin (requires admin role)")
        gr.Markdown("You must be logged in as an admin user to see details here.")

        with gr.Row():
            refresh_stats_btn = gr.Button("DB Stats")
            refresh_users_btn = gr.Button("List Users")
            refresh_cohorts_btn = gr.Button("List Cohorts")
            refresh_audit_btn = gr.Button("Audit Log (last 50)")

        admin_stats_md = gr.Markdown()
        admin_users_md = gr.Markdown()
        admin_cohorts_md = gr.Markdown()
        admin_audit_md = gr.Markdown()

        gr.Markdown("#### Delete Cohort (admin only)")
        admin_delete_cohort_dropdown = gr.Dropdown(
            label="Select Cohort to Delete",
            choices=[],
        )
        admin_delete_btn = gr.Button("Delete Selected Cohort")
        admin_delete_status_md = gr.Markdown()

        def _ensure_admin(session_user):
            if not session_user:
                return None, "❌ Not logged in."
            username = session_user.get("username")
            role = session_user.get("role", "user")
            su = SessionUser(username=username, role=role)
            if not su.is_admin:
                return None, "❌ Admin privileges required."
            return su, ""

        # --- NEW helper: ensures only one admin pane is populated at a time ---
        def _admin_pack(active: str, text: str):
            """
            active in {"stats","users","cohorts","audit"}
            Returns 4 outputs in the order: stats, users, cohorts, audit
            """
            stats = text if active == "stats" else ""
            users = text if active == "users" else ""
            cohorts = text if active == "cohorts" else ""
            audit = text if active == "audit" else ""
            return stats, users, cohorts, audit

        def _refresh_stats(session_user):
            su, msg = _ensure_admin(session_user)
            if not su:
                return _admin_pack("stats", msg)
            try:
                return _admin_pack("stats", get_db_stats())
            except Exception as e:
                trace_log(f"ADMIN _refresh_stats ERROR: {e}")
                return _admin_pack("stats", f"❌ Error loading DB stats: {e}")

        def _refresh_users(session_user):
            su, msg = _ensure_admin(session_user)
            if not su:
                return _admin_pack("stats", msg)
            try:
                return _admin_pack("users", describe_users())
            except Exception as e:
                trace_log(f"ADMIN _refresh_users ERROR: {e}")
                return _admin_pack("users", f"❌ Error loading users: {e}")

        # Split cohorts refresh into:
        #   (A) markdown panes (4 outputs)
        #   (B) delete dropdown choices (1 output)
        def _refresh_cohorts_admin_md(session_user):
            su, msg = _ensure_admin(session_user)
            if not su:
                return _admin_pack("stats", msg)
            try:
                return _admin_pack("cohorts", describe_cohorts())
            except Exception as e:
                trace_log(f"ADMIN _refresh_cohorts_admin_md ERROR: {e}")
                return _admin_pack("cohorts", f"❌ Error loading cohorts: {e}")

        def _refresh_cohorts_admin_dd(session_user):
            su, msg = _ensure_admin(session_user)
            if not su:
                return gr.update(choices=[], value=None)
            try:
                names = [row[0] for row in get_all_cohorts()]
                return gr.update(choices=names, value=None)
            except Exception as e:
                trace_log(f"ADMIN _refresh_cohorts_admin_dd ERROR: {e}")
                return gr.update(choices=[], value=None)

        def _refresh_audit(session_user):
            su, msg = _ensure_admin(session_user)
            if not su:
                return _admin_pack("stats", msg)
            try:
                return _admin_pack("audit", admin_view_audit_log(su, limit=50))
            except Exception as e:
                trace_log(f"ADMIN _refresh_audit ERROR: {e}")
                return _admin_pack("audit", f"❌ Error loading audit log: {e}")

        def _delete_cohort_cb(session_user, cohort_name: str):
            su, msg = _ensure_admin(session_user)
            if not su:
                return msg
            if not cohort_name:
                return "❌ Please select a cohort."
            try:
                return admin_delete_cohort(su, cohort_name)
            except Exception as e:
                trace_log(f"ADMIN _delete_cohort_cb ERROR: {e}")
                return f"❌ Error deleting cohort: {e}"

        # --- REWIRED: each button updates ALL FOUR panes ---
        refresh_stats_btn.click(
            _refresh_stats,
            inputs=[session_state],
            outputs=[admin_stats_md, admin_users_md, admin_cohorts_md, admin_audit_md],
        )

        refresh_users_btn.click(
            _refresh_users,
            inputs=[session_state],
            outputs=[admin_stats_md, admin_users_md, admin_cohorts_md, admin_audit_md],
        )

        # Cohorts button: two handlers (panes + dropdown)
        refresh_cohorts_btn.click(
            _refresh_cohorts_admin_md,
            inputs=[session_state],
            outputs=[admin_stats_md, admin_users_md, admin_cohorts_md, admin_audit_md],
        )

        refresh_cohorts_btn.click(
            _refresh_cohorts_admin_dd,
            inputs=[session_state],
            outputs=[admin_delete_cohort_dropdown],
        )

        refresh_audit_btn.click(
            _refresh_audit,
            inputs=[session_state],
            outputs=[admin_stats_md, admin_users_md, admin_cohorts_md, admin_audit_md],
        )

        admin_delete_btn.click(
            _delete_cohort_cb,
            inputs=[session_state, admin_delete_cohort_dropdown],
            outputs=[admin_delete_status_md],
        )


        # --- Demo Setup (v16.1) ---
        gr.Markdown("### Demo Setup (admin)")
        gr.Markdown("Upload a prompt list to preload curated 'demo prompts' for a cohort. These appear in the Ask tab as a dropdown when that cohort is selected.")

        with gr.Row():
            demo_cohort_dropdown = gr.Dropdown(label="Cohort for Demo Prompts", choices=[], value=None)
            demo_refresh_cohorts_btn = gr.Button("Refresh Cohorts")

        with gr.Row():
            demo_prompts_file = gr.File(
                label="Upload Demo Prompts File (.txt, .csv, .json)",
                file_count="single",
                type="filepath",
            )
            demo_replace_checkbox = gr.Checkbox(label="Replace existing demo prompts", value=True)

        with gr.Row():
            demo_load_btn = gr.Button("Load Demo Prompts", variant="primary")
            demo_view_btn = gr.Button("View Demo Prompts")
            demo_clear_btn = gr.Button("Clear Demo Prompts")

        demo_prompts_status = gr.Markdown("")
        demo_prompts_preview = gr.Markdown("")

        def _demo_refresh_cohorts_dd(session_user):
            su, msg = _ensure_admin(session_user)
            if not su:
                return gr.update(choices=[], value=None)
            try:
                rows = get_all_cohorts()
                names = [r[0] for r in rows if r and r[0]]
                return gr.update(choices=names, value=None)
            except Exception as e:
                trace_log(f"ADMIN demo _demo_refresh_cohorts_dd ERROR: {e}")
                return gr.update(choices=[], value=None)

        def _demo_load_prompts_cb(session_user, cohort_name: str, file_path: str, replace: bool):
            su, msg = _ensure_admin(session_user)
            if not su:
                return msg or "❌ Admin privileges required.", ""
            c = (cohort_name or "").strip()
            if not c:
                return "❌ Please select a cohort.", ""
            if not file_path:
                return "❌ Please upload a .txt/.csv/.json prompt file.", ""
            prompts = parse_demo_prompts_file(file_path)
            n = upsert_demo_prompts_for_cohort(c, prompts, replace=bool(replace))
            try:
                log_audit(su.username, su.role, "demo_prompts_load", f"cohort={c} count={n} replace={bool(replace)}")
            except Exception as e:
                trace_log(f"log_audit demo_prompts_load ERROR: {e}")
            preview = list_demo_prompts_for_cohort(c)
            preview_md = "\n".join([f"- {p}" for p in preview[:50]]) if preview else "_No demo prompts found._"
            return f"✅ Loaded {n} demo prompt(s) into cohort `{c}`.", preview_md

        def _demo_view_prompts_cb(session_user, cohort_name: str):
            su, msg = _ensure_admin(session_user)
            if not su:
                return msg or "❌ Admin privileges required.", ""
            c = (cohort_name or "").strip()
            if not c:
                return "❌ Please select a cohort.", ""
            preview = list_demo_prompts_for_cohort(c)
            if not preview:
                return f"_No demo prompts found for `{c}`._", ""
            md = "\n".join([f"- {p}" for p in preview[:200]])
            return f"**Demo prompts for `{c}` ({len(preview)}):**", md

        def _demo_clear_prompts_cb(session_user, cohort_name: str):
            su, msg = _ensure_admin(session_user)
            if not su:
                return msg or "❌ Admin privileges required.", ""
            c = (cohort_name or "").strip()
            if not c:
                return "❌ Please select a cohort.", ""
            clear_demo_prompts_for_cohort(c)
            try:
                log_audit(su.username, su.role, "demo_prompts_clear", f"cohort={c}")
            except Exception as e:
                trace_log(f"log_audit demo_prompts_clear ERROR: {e}")
            return f"✅ Cleared demo prompts for `{c}`.", ""

        demo_refresh_cohorts_btn.click(
            _demo_refresh_cohorts_dd,
            inputs=[session_state],
            outputs=[demo_cohort_dropdown],
        )

        demo_load_btn.click(
            _demo_load_prompts_cb,
            inputs=[session_state, demo_cohort_dropdown, demo_prompts_file, demo_replace_checkbox],
            outputs=[demo_prompts_status, demo_prompts_preview],
        )

        demo_view_btn.click(
            _demo_view_prompts_cb,
            inputs=[session_state, demo_cohort_dropdown],
            outputs=[demo_prompts_status, demo_prompts_preview],
        )

        demo_clear_btn.click(
            _demo_clear_prompts_cb,
            inputs=[session_state, demo_cohort_dropdown],
            outputs=[demo_prompts_status, demo_prompts_preview],
        )

        return {
            "admin_stats_md": admin_stats_md,
            "admin_users_md": admin_users_md,
            "admin_cohorts_md": admin_cohorts_md,
            "admin_audit_md": admin_audit_md,
            "admin_delete_cohort_dropdown": admin_delete_cohort_dropdown,
            "admin_delete_status_md": admin_delete_status_md,
        }

    ACTION_CREATE = "Create new cohort from files"
    ACTION_APPEND = "Append files to existing cohort"
    ACTION_CLONE  = "Clone an existing cohort"

    DEFAULT_ACTION = ACTION_CREATE



    # -----------------------------
    # Build Gradio Interface
    # -----------------------------
    def build_interface():
        with gr.Blocks(title="Phase1 RAG MVP v16_1") as demo:
            gr.Markdown("## Phase1 RAG MVP v16_1")

            # Global session state
            session_state = gr.State(value=None)       # {"username": ..., "role": ...} or None
            role_state = gr.State(value="anonymous")   # Optional, not heavily used yet
            api_key_state = gr.State(value="")         # Hidden backing store for API key

            # ---- Login / Logout row ----
            with gr.Row():
                username_tb = gr.Textbox(label="Username", scale=1)
                password_tb = gr.Textbox(label="Password", type="password", scale=1)
                api_key_tb = gr.Textbox(
                    label="OpenAI API Key",
                    type="password",
                    placeholder="sk-...",
                    scale=2,
                )

            with gr.Row():
                login_btn = gr.Button("Login")
                logout_btn = gr.Button("Logout")

            login_status = gr.Markdown("Not logged in.")
            current_user_label = gr.Markdown("**Current user:** (none)")

            # ---- Tabs ----
            with gr.Tabs():
                with gr.Tab("Setup & Cohorts"):
                    setup_handles = build_setup_tab(session_state, api_key_state)
                with gr.Tab("Ask"):
                    ask_handles = build_ask_tab(session_state, api_key_state)
                with gr.Tab("History"):
                    history_handles = build_history_tab(session_state)
                # Admin tab: hidden by default
                admin_tab = gr.Tab("Admin", visible=False)
                with admin_tab:
                    admin_handles = build_admin_tab(session_state)
            # ---- Ask tab enable / disable JS (Fix Set 1) ----
            ASK_DISABLE_JS = """
            (session_state, role_state, api_key_state) => {
              const setDisabled = (containerId, disabled) => {
                const container = document.getElementById(containerId);
                if (!container) return;
                const el = container.querySelector("textarea, input, button, select");
                if (!el) return;
                el.disabled = disabled;
              };

              setDisabled("ask_cohort_dropdown", true);
              setDisabled("ask_model_dropdown", true);
              setDisabled("ask_question_box", true);
              setDisabled("ask_improved_box", true);
              setDisabled("ask_improve_btn", true);
              setDisabled("ask_ask_btn", true);
              setDisabled("ask_refresh_btn", true);

              // CRITICAL: return inputs unchanged
              return [session_state, role_state, api_key_state];
            }
            """


            ASK_ENABLE_JS = """
            (username, password, api_key, session_state, role_state, api_key_state) => {
              const setDisabled = (containerId, disabled) => {
                const container = document.getElementById(containerId);
                if (!container) return;
                const el = container.querySelector("textarea, input, button, select");
                if (!el) return;
                el.disabled = disabled;
              };

              setDisabled("ask_cohort_dropdown", false);
              setDisabled("ask_model_dropdown", false);
              setDisabled("ask_question_box", false);
              setDisabled("ask_improved_box", false);
              setDisabled("ask_improve_btn", false);
              setDisabled("ask_ask_btn", false);
              setDisabled("ask_refresh_btn", false);

              // CRITICAL: return inputs unchanged so Python receives them
              return [username, password, api_key, session_state, role_state, api_key_state];
            }
            """




            # ---- Login / Logout Callbacks ----
            # ---- Login / Logout Callbacks ----

            def _login_cb(username, password, api_key, current_session, current_role, current_api_key):
                """
                Authenticate user and store session + API key in state.
                Also auto-populates cohort dropdowns for this user.

                POLICY:
                  - API key is stored only in api_key_state (memory), never left in a textbox.
                  - On ANY login attempt (success or failure), clear:
                      Username, Password, API Key textbox, Question, Improved Prompt
                """
                # print("LOGIN RAW:", repr(username), repr(password), repr(api_key))

                #print("DEBUG: login clicked")

                # Normalize inputs
                username = (username or "").strip()
                password = password or ""
                api_key = (api_key or "").strip()

                # Always-clear values for UI textboxes (return plain strings for reliability)
                CLEAR = ""
                clear_username = CLEAR
                clear_password = CLEAR
                clear_api_key_tb = CLEAR
                clear_question = CLEAR
                clear_improved = CLEAR

                # Basic validation
                if not username or not password:
                    return (
                        "❌ Username and password are required.",  # login_status
                        None,                                     # session_state
                        "anonymous",                               # role_state
                        "",                                       # api_key_state
                        clear_username,                            # username_tb
                        clear_password,                            # password_tb
                        clear_api_key_tb,                          # api_key_tb
                        "**Current user:** (none)",                # current_user_label
                        gr.update(choices=[], value=None),         # setup existing cohort dropdown
                        gr.update(choices=[], value=None),         # ask cohort dropdown
                        clear_question,                            # ask question_box
                        clear_improved,                            # ask improved_box
                    )

                user_dict = authenticate_credentials(username, password)
                if not user_dict:
                    return (
                        "❌ Invalid username or password.",         # login_status
                        None,                                     # session_state
                        "anonymous",                               # role_state
                        "",                                       # api_key_state
                        clear_username,                            # username_tb
                        clear_password,                            # password_tb
                        clear_api_key_tb,                          # api_key_tb
                        "**Current user:** (none)",                # current_user_label
                        gr.update(choices=[], value=None),         # setup existing cohort dropdown
                        gr.update(choices=[], value=None),         # ask cohort dropdown
                        clear_question,                            # ask question_box
                        clear_improved,                            # ask improved_box
                        gr.update(visible=False)   # admin_tab

                    )

                role = user_dict.get("role", "user")
                status = f"✅ Logged in as `{username}` (role: `{role}`)"
                label = f"**Current user:** `{username}` (role: `{role}`)"

                # Store API key ONLY in state (memory); do not leave it in textbox
                new_api_key_state = api_key

                # Populate cohorts for this user (respects sharing rules)
                try:
                    cohort_names = get_cohorts_for_user(username)
                except Exception as e:
                    trace_log(f"LOGIN get_cohorts_for_user ERROR: {e}")
                    try:
                        cohort_names = list_cohorts()
                    except Exception:
                        cohort_names = []

                # Build "name/owner" labels for the Setup tab dropdown
                try:
                    rows = get_all_cohorts()  # (cohort_name, owner_user_id, ...)
                    owner_by_name = {row[0]: (row[1] if len(row) > 1 else None) for row in rows}
                    setup_labels = []
                    for name in cohort_names:
                        owner = owner_by_name.get(name)
                        setup_labels.append(f"{name}/{owner}" if owner else name)
                except Exception as e:
                    trace_log(f"LOGIN building setup_labels ERROR: {e}")
                    setup_labels = cohort_names

                setup_existing_dd = gr.update(choices=setup_labels, value=None)
                ask_cohort_dd = gr.update(choices=cohort_names, value=None)

                return (
                    status,                 # login_status
                    user_dict,              # session_state
                    role,                   # role_state
                    new_api_key_state,      # api_key_state
                    clear_username,         # username_tb (cleared after login)
                    clear_password,         # password_tb (cleared after login)
                    clear_api_key_tb,       # api_key_tb (cleared after login)
                    label,                  # current_user_label
                    setup_existing_dd,      # setup existing cohort dropdown
                    ask_cohort_dd,          # ask cohort dropdown
                    clear_question,         # ask question_box (cleared after login)
                    clear_improved,# ask improved_box (cleared after login)
                    gr.update(visible=(role == "admin")),  # NEW admin_tab visibility
                )


            login_btn.click(
                _login_cb,
                inputs=[
                    username_tb,
                    password_tb,
                    api_key_tb,
                    session_state,
                    role_state,
                    api_key_state,
                ],
                queue=False,
                js=ASK_ENABLE_JS,

                outputs=[
                    login_status,
                    session_state,
                    role_state,
                    api_key_state,
                    username_tb,
                    password_tb,
                    api_key_tb,
                    current_user_label,
                    setup_handles["existing_cohort_dropdown"],
                    ask_handles["cohort_dropdown"],
                    ask_handles["question_box"],
                    ask_handles["improved_box"],
                    admin_tab,   # NEW
                ],

            )

            def _logout_cb(_session, _role, _api_key):
                # Reset in-memory state
                new_session = SessionUser(username=None, role="guest")
                new_role = "guest"
                new_api_key = ""

                # Use plain strings for Textbox clearing (more reliable than gr.update for Textbox values)
                CLEAR = ""

                reset_files = gr.update(value=None)
                reset_radio = gr.update(value=None)
                reset_dd_empty = gr.update(value=None, choices=[])

                # IMPORTANT: model dropdown must keep valid choices
                reset_model_dd = gr.update(choices=list_chat_models(), value=None)

                # History filters (keep simple defaults; prevents stale selections)
                reset_history_user = gr.update(value="")
                reset_history_cohort = gr.update(value="")


                return (
                    "🔒 Logged out.",           # login_status
                    new_session,               # session_state
                    new_role,                  # role_state
                    new_api_key,               # api_key_state

                    CLEAR,                     # username_tb
                    CLEAR,                     # password_tb
                    CLEAR,                     # api_key_tb
                    "Not logged in",           # current_user_label

                    reset_radio,               # setup action_radio
                    CLEAR,                     # setup new_cohort_name
                    gr.update(value=False),    # setup share_checkbox
                    reset_dd_empty,            # setup existing_cohort_dropdown
                    reset_files,               # setup file_uploader
                    CLEAR,                     # setup build_status (markdown/text is fine as "")

                    reset_dd_empty,            # ask cohort_dropdown
                    reset_model_dd,            # ask model_dropdown  (DO NOT empty choices)
                    CLEAR,                     # ask question_box
                    CLEAR,                     # ask improved_box
                    CLEAR,                     # ask answer_markdown
                    CLEAR,                     # ask docs_markdown

                    reset_history_user,        # history_user_filter
                    reset_history_cohort,      # history_cohort_filter
                    CLEAR,                     # history_markdown

                    CLEAR,                     # admin_stats_md
                    CLEAR,                     # admin_users_md
                    CLEAR,                     # admin_cohorts_md
                    CLEAR,                     # admin_audit_md
                    reset_dd_empty,            # admin_delete_cohort_dropdown
                    CLEAR,                      # admin_delete_status_md
                    gr.update(visible=False)   # admin_tab hidden after logout

                )



            logout_btn.click(
                _logout_cb,
                inputs=[session_state, role_state, api_key_state],
                outputs=[
                    login_status,
                    session_state,
                    role_state,
                    api_key_state,
                    username_tb,
                    password_tb,
                    api_key_tb,
                    current_user_label,

                    setup_handles["action_radio"],
                    setup_handles["new_cohort_name"],
                    setup_handles["share_checkbox"],
                    setup_handles["existing_cohort_dropdown"],
                    setup_handles["file_uploader"],
                    setup_handles["build_status"],

                    ask_handles["cohort_dropdown"],
                    ask_handles["model_dropdown"],
                    ask_handles["question_box"],
                    ask_handles["improved_box"],
                    ask_handles["answer_markdown"],
                    ask_handles["docs_markdown"],

                    history_handles["history_user_filter"],
                    history_handles["history_cohort_filter"],
                    history_handles["history_markdown"],

                    admin_handles["admin_stats_md"],
                    admin_handles["admin_users_md"],
                    admin_handles["admin_cohorts_md"],
                    admin_handles["admin_audit_md"],
                    admin_handles["admin_delete_cohort_dropdown"],
                    admin_handles["admin_delete_status_md"],
                    admin_tab,   # NEW
                ],
                queue=False,
                js=ASK_DISABLE_JS,

            )

        return demo

    # ---- Create & Launch the App ----
    if __name__ == "__main__":
        demo = build_interface()
        # In Colab: share=False is usually fine; set to True if you want a public link.
        demo.launch(share=False, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
